<a href="https://colab.research.google.com/github/softcomput/cp-colab/blob/main/CPP_To_Work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ortools

In [ ]:
import logging
import re

from ortools.linear_solver import pywraplp
import numpy as np


class _CPPToWork:
    """_CPPToWork

    Attributes:
        data (dict):
            Vs (list): servers
            Vc (list): clients
            Q (list): capacities
            T (list): maximum ride times
            Ls (list): latest arrival time of servers
            Lc (list): latest arrival time of clients
            distance (2-d list): distance for each server and client
            duration (2-d list): duration for each server and client
    """
    def __init__(self, data):
        self.d = self.__format_data(data)

    def __format_data(self, data):
        """ __format_data

        Format data model.
        """
        distance = []
        for vs_d in data['distance'][0:len(data['Vs'])]:
            distance.append(vs_d)
            distance.extend(data['distance'][len(data['Vs']):])
        data['distance'] = distance

        duration = []
        for vs_d in data['duration'][0:len(data['Vs'])]:
            duration.append(vs_d)
            duration.extend(data['duration'][len(data['Vs']):])
        data['duration'] = duration

        duration_vc = []
        for vc in range(len(data['Vc'])):
            duration_vc.append(data['duration'][vc + 1][0])
        data['duration_vc'] = duration_vc

        l_vc = []
        for vs in range(len(data['Vs'])):
            l = []
            l.append(data['Ls'][vs])
            l.extend(data['Lc'])
            l_vc.append(l)
        data['L'] = l_vc

        if data.get('preference'):
            preference = []
            for vs_d in data['preference'][0:len(data['Vs'])]:
                preference.append(vs_d)
                preference.extend(data['preference'][len(data['Vs']):])
            data['preference'] = preference

        return data

    def __create_variables(self, solver):
        """ __create_variables

        Create variables.

        Returns:
            x (list): x^k_ij, k in Vs
            y (list): y_i, i in Vc
            u (list): u^k_i, 1 <= u^k_i <= L_i - t_i0, i in V
        """
        x = []
        u = []
        for idx, k in enumerate(self.d['Vs']):
            ai = [k]
            ai.extend(self.d['Vc'])
            aj = [0]
            aj.extend(self.d['Vc'])
            for i in ai:
                x.append([
                    solver.IntVar(0, 1, 'x{}({},{})'.format(k, i, j))
                    for j in aj])

            t_i0 = [self.d['duration'][(1 + len(self.d['Vc'])) * idx][0]]
            for idx_i in range(len(ai[1:])):
                t_i0.append(self.d['duration'][
                    1 + (1 + len(self.d['Vc'])) * idx + idx_i][0])
            u.append([solver.IntVar(
                1,
                self.d['L'][idx][idx_i] - t_i0[idx_i],
                'u{}({})'.format(k, i)) for idx_i, i in enumerate(ai)])
        y = [solver.IntVar(0, 1, 'y({})'.format(i)) for i in self.d['Vc']]
        return x, y, u

    def __create_constraints(self, solver, x, y, u):
        """ __create_constraints

        Create constraints.
        """
        n = len(self.d['Vc']) + 1

        for vs in range(len(self.d['Vs'])):
            k = vs * n
            # Each employee leaves its house.
            solver.Add(solver.Sum(x[k]) == 1)
            # Each employee arrives at the workspace.
            solver.Add(solver.Sum([x[j][0] for j in range(k, k + n)]) == 1)
            # Continuity.
            for i in range(1, len(self.d['Vc']) + 1):
                solver.Add(
                    solver.Sum([x[j][i] for j in range(k, k + n)]) -
                    solver.Sum(x[k + i]) == 0)
            # Capacity.
            solver.Add(
                solver.Sum([x_k for i in range(n) for x_k in x[k + i]]) <=
                self.d['Q'][vs])
            # Maximum ride time.
            solver.Add(
                solver.Sum([np.inner(d, _x) for d, _x in zip(
                    self.d['duration'][k:k + n], x[k:k + n])]) <=
                self.d['T'][vs])
            # Own arc is not passed.
            for i in range(1, len(self.d['Vc']) + 1):
                solver.Add(x[k + i][i] == 0)
            # Arrival time condition with subtour elimination constraints.
            for j in range(1, len(u[vs])):
                solver.Add(
                    u[vs][0] +
                    self.d['duration'][k][j] -
                    self.d['Ls'][vs] * (1 - x[k][j]) <=
                    u[vs][j])
            for i in range(1, len(u[vs])):
                for j in range(1, len(u[vs])):
                    if str(u[vs][i]) != str(u[vs][j]):
                        solver.Add(
                            u[vs][i] +
                            self.d['duration'][k + i][j] -
                            self.d['Ls'][vs] * (1 - x[k + i][j]) <=
                            u[vs][j])
            solver.Add(u[vs][0] == 1)

        # Each client is either picked up by a server or is left unserviced.
        for vc in range(len(self.d['Vc'])):
            solver.Add(
                solver.Sum([x_k for vs in range(len(self.d['Vs']))
                           for x_k in x[vs * n + vc + 1]]) + y[vc] == 1)

    def __create_objective(self, solver, x, y):
        """ __create_objective

        Create objective.
        """
        if self.d.get('preference'):
            solver.Minimize(
                solver.Sum([np.inner(d, _x) for d, _x in zip(
                    np.array(self.d['preference']) *
                    np.array(self.d['duration']), x)]) +
                solver.Sum([np.inner(np.array(self.d['duration_vc']) * 1, y)]))
        else:
            solver.Minimize(
                solver.Sum([np.inner(d, _x) for d, _x in zip(
                    self.d['duration'], x)]) +
                solver.Sum([np.inner(np.array(self.d['duration_vc']) * 2, y)]))

    def solve(self):
        """ solve

        Solve CPP.
        """
        solver = pywraplp.Solver(
            'cp', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

        x, y, u = self.__create_variables(solver)
        self.__create_constraints(solver, x, y, u)
        self.__create_objective(solver, x, y)

        status = solver.Solve()

        paths = {}
        if status == pywraplp.Solver.OPTIMAL:
            n = len(self.d['Vc']) + 1
            root = []
            for i in range(len(self.d['Vs']) * n):
                for j in range(n):
                    if x[i][j].solution_value():
                        root.append(x[i][j].name())
                        r = re.findall(r'\w+', x[i][j].name())
                        r[0] = r[0].lstrip('x')
                        if not r[0] in paths:
                            paths[r[0]] = []
                        paths[r[0]].append((r[1], r[2]))
            paths = self.__format_paths(paths)
            not_pickup_clients = []
            for i in range(len(self.d['Vc'])):
                not_pickup_clients.append((y[i].name(), y[i].solution_value()))
            arrival_times = []
            for i in range(len(self.d['Vs'])):
                arrival_times.append([
                    (u[i][j].name(), u[i][j].solution_value())
                    for j in range(len(u[i]))])
            logging.info(
                'Root %s, not pickup clients %s, and arrival_times %s solved in %d variables, \
                %d constraints, %f objective value, %f milliseconds, \
                %d iterations, and %d branch-and-bound nodes.' % (
                    root, not_pickup_clients, arrival_times,
                    solver.NumVariables(),
                    solver.NumConstraints(),
                    solver.Objective().Value(),
                    solver.wall_time(), solver.iterations(), solver.nodes()))
        else:
            logging.info('The problem does not have an optimal solution.')
        m, tdr, rti = self.__evaluate(x, y)
        return {
            'feasible_paths': paths,
            '%m': m,
            '%tdr': tdr,
            '%rti': rti
        }

    def __format_paths(self, paths):
        for i in paths:
            clients = []
            next = paths[i][0][1]
            for _ in paths[i]:
                if not next == '0':
                    for j in paths[i]:
                        if next == j[0]:
                            clients.append(j[0])
                            next = j[1]
                            break
            paths[i] = clients
        return paths

    def __evaluate(self, x, y):
        total_distance = 0
        total_duration = 0
        original_distance = 0
        original_duration_vs = 0
        n = len(self.d['Vc']) + 1
        for i in range(len(self.d['Vs']) * n):
            for j in range(n):
                if x[i][j].solution_value():
                    total_distance += self.d['distance'][i][j]
                    total_duration += self.d['duration'][i][j]
        for i in range(len(self.d['Vs'])):
            original_distance += self.d['distance'][i * n][0]
            original_duration_vs += self.d['duration'][i * n][0]
        not_pickup_clients = 0
        for i in range(len(self.d['Vc'])):
            if y[i].solution_value() == 1:
                not_pickup_clients += 1
                total_distance += self.d['distance'][i + 1][0]
            original_distance += self.d['distance'][i + 1][0]

        m = (len(self.d['Vc']) - not_pickup_clients) / len(self.d['Vc'])
        tdr = 1 - total_distance / original_distance
        rti = total_duration / original_duration_vs
        return m, tdr, rti

data = create_data_model()
output = _CPPToWork(data).solve()
print(output)

{'feasible_paths': {'3': ['205', '242', '196'], '5': ['190', '282'], '13': ['249', '208', '237', '264'], '14': ['222'], '17': ['157'], '26': [], '29': ['274', '168', '301'], '40': ['235'], '41': ['177'], '44': ['245', '229']}, '%m': 1.0, '%tdr': 0.48038629276252753, '%rti': 1.3573770491803279}


In [ ]:
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['Vs'] = ['a', 'b']
    data['Vc'] = ['c', 'd', 'e', 'f', 'g', 'h', 'i']
    data['Q'] = [7, 5]
    data['T'] = [12, 15]
    data['Ls'] = [120, 120]
    data['Lc'] = [60, 60, 60, 60, 60, 60, 101]
    data['distance'] = [
                        [10, 5, 7, 10, 1, 3, 10, 100],
                        [10, 7, 9, 4, 11, 11, 12, 100],
                        [5, 0, 2, 5, 4, 2, 5, 100],
                        [3, 2, 0, 5, 6, 4, 3, 100],
                        [6, 5, 5, 0, 9, 7, 8, 100],
                        [9, 4, 6, 9, 0, 2, 9, 100],
                        [7, 2, 4, 7, 2, 0, 7, 100],
                        [1, 5, 3, 8, 9, 7, 0, 100],
                        [100, 100, 100, 100, 100, 100, 100, 0],
                        ]
    data['duration'] = [
                        [10, 5, 7, 10, 1, 3, 10, 100],
                        [10, 7, 9, 4, 11, 11, 12, 100],
                        [5, 0, 2, 5, 4, 2, 5, 100],
                        [3, 2, 0, 5, 6, 4, 3, 100],
                        [6, 5, 5, 0, 9, 7, 8, 100],
                        [9, 4, 6, 9, 0, 2, 9, 100],
                        [7, 2, 4, 7, 2, 0, 7, 100],
                        [1, 5, 3, 8, 9, 7, 0, 100],
                        [100, 100, 100, 100, 100, 100, 100, 0],
                        ]
    # data['preference'] = [
    #                     [1, 1, 1, 1, 1, 1, 1, 1],
    #                     [1, 1, 1, 1, 1, 1, 0.1, 1],
    #                     [1, 0, 1, 1, 1, 1, 1, 1],
    #                     [1, 1, 0, 1, 1, 1, 1, 1],
    #                     [1, 1, 1, 0, 1, 1, 0.1, 1],
    #                     [1, 1, 1, 1, 0, 1, 1, 1],
    #                     [1, 1, 1, 1, 1, 0, 1, 1],
    #                     [1, 1, 1, 0.1, 1, 1, 0, 1],
    #                     [1, 1, 1, 1, 1, 1, 1, 0],
    #                     ]
    return data

In [ ]:
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['Vs'] = ['3', '5', '13', '14', '17', '26', '29', '40', '41', '44']
    data['Vc'] = ['157', '160', '168', '177', '190', '196', '205', '208', '222', '229', '235', '237', '242', '245', '248', '249', '264', '274', '282', '301']
    data['Q'] = [5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
    data['T'] = [3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600]
    data['Ls'] = [7200, 7200, 7200, 7200, 7200, 7200, 7200, 7200, 7200, 7200]
    data['Lc'] = [7200, 7200, 7200, 7200, 7200, 7200, 7200, 7200, 7200, 7200, 7200, 7200, 7200, 7200, 7200, 7200, 7200, 7200, 7200, 7200]
    data['distance'] = [[5945,5095,8433,4113,4080,2195,3817,408,7770,4305,7509,3146,5903,1784,7159,8433,8189,4936,6153,4225,2625],
                        [6536,3977,9076,3358,4401,2318,4408,922,7015,6567,8670,3467,5148,2501,7750,9076,8432,5191,5398,4546,2880],
                        [6043,12573,1788,6739,7149,7045,6547,8618,4219,10344,3351,7244,4181,7374,3071,1788,2665,5704,5164,6714,6247],
                        [3824,8453,6311,4339,1418,2930,1696,4432,7132,2075,5388,1015,5287,2220,5038,6311,7491,3496,6379,2725,2672],
                        [8022,3780,9733,4137,5058,3183,5894,2175,7672,7224,9327,4124,5805,3366,9047,9733,9089,5848,5864,5203,3537],
                        [8202,5281,10153,7553,5940,4577,5537,3749,10147,5547,10224,5356,8302,3693,9874,10153,10506,7253,9593,6303,5547],
                        [7623,9725,5969,4522,5424,5320,6819,6808,1588,8619,6047,5519,1454,5649,5697,5969,3005,3979,2553,4989,3822],
                        [7209,4661,8665,2947,3990,2412,5777,1815,6604,6156,8259,3056,4737,2717,7979,8665,8021,4780,4987,4135,2469],
                        [4129,7785,5773,3671,605,2262,2765,3764,6008,3339,5367,978,4163,2590,4891,5773,6367,1601,5711,1060,1392],
                        [6007,13056,1752,7222,7632,7528,6511,9101,4702,8677,3315,7727,4664,7857,3035,1752,3148,6187,5647,7197,6730],
                        [11788,0,23024,7029,8071,6494,9123,5230,10686,9133,13810,7137,8819,7279,13460,23024,12103,8862,9069,8216,6551],
                        [4727,23024,0,7096,5859,7509,5231,9082,5681,7397,2035,6422,5315,7738,1755,0,4127,4609,6626,5206,6711],
                        [5698,7029,7096,0,4224,3460,4573,4074,4875,6390,6526,3290,2686,3915,6246,7311,6292,3075,2320,3007,1694],
                        [4067,8071,5859,4224,0,2236,1939,4344,6311,2994,4747,1558,4466,3334,4397,5706,6670,1534,5501,993,1682],
                        [4988,6494,7509,3460,2236,0,3185,2377,5738,4148,6425,1048,3893,1154,6145,6831,6097,2844,4957,1894,1138],
                        [2238,9123,5231,4573,1939,3185,0,4201,7534,2766,4260,2691,5689,2931,3910,5039,7222,2758,7017,2169,2906],
                        [6199,5230,9082,4074,4344,2377,4201,0,7634,4559,7763,3010,5767,1986,7413,8499,7765,4512,6017,4089,2489],
                        [8812,10686,5681,4875,6311,5738,7534,7634,0,9485,6760,6385,2320,6515,6410,6184,1417,4845,3304,5855,4688],
                        [4177,9133,7397,6390,2994,4148,2766,4559,9485,0,6199,3093,7365,3429,5849,7205,9569,4440,8457,3899,4750],
                        [3375,13810,2035,6526,4747,6425,4260,7763,6760,6199,0,6789,5050,7086,350,2890,5189,3872,6753,5958,6694],
                        [4842,7137,6422,3290,1558,1048,2691,3010,6385,3093,6789,0,4264,2236,5547,6233,6468,2474,5357,1703,1649],
                        [6595,8819,5315,2686,4466,3893,5689,5767,2320,7365,5050,4264,0,4621,4669,5664,3592,2951,1699,3961,3353],
                        [5760,7279,7738,3915,3334,1154,2931,1986,6515,3429,7086,2236,4621,0,6916,7602,6868,3615,5729,2665,1910],
                        [3025,13460,1755,6246,4397,6145,3910,7413,6410,5849,350,5547,4669,6916,0,2540,4839,3522,6403,5608,6344],
                        [4727,23024,0,7311,5706,6831,5039,8499,6184,7205,2890,6233,5664,7602,2540,0,4127,4609,6626,5206,6711],
                        [7118,12103,4127,6292,6670,6097,7222,7765,1417,9569,5189,6468,3592,6868,4839,4127,0,5067,4721,6077,5610],
                        [3956,8862,4609,3075,1534,2844,2758,4512,4845,4440,3872,2474,2951,3615,3522,4609,5067,0,4394,863,1966],
                        [8094,9069,6626,2320,5501,4957,7017,6017,3304,8457,6753,5357,1699,5729,6403,6626,4721,4394,0,5460,3760],
                        [3783,8216,5206,3007,993,1894,2169,4089,5855,3899,5958,1703,3961,2665,5608,5206,6077,863,5460,0,1608],
                        [4692,6551,6711,1694,1682,1138,2906,2489,4688,4750,6694,1649,3353,1910,6344,6711,5610,1966,3760,1608,0]]
    data['duration'] = [[1027,707,1332,686,582,374,602,132,1157,605,1239,387,966,315,1149,1332,1208,784,960,667,457],
                        [1130,539,1349,537,541,362,705,216,1008,646,1250,346,817,349,1252,1349,1150,784,811,626,458],
                        [1113,1742,394,1045,1100,1039,1090,1303,570,1316,642,1016,652,1067,563,394,414,812,780,1041,963],
                        [701,934,1006,550,184,292,276,491,995,226,913,74,829,289,823,1006,993,538,824,399,341],
                        [1288,565,1467,632,659,499,863,406,1125,764,1368,464,934,486,1289,1467,1267,902,916,744,576],
                        [1049,496,1447,1011,746,531,717,513,1346,658,1294,571,1180,445,1204,1447,1343,919,1285,862,712],
                        [1311,1433,839,681,983,922,1127,1113,247,1199,1059,899,304,950,969,839,388,695,379,924,812],
                        [1126,660,1342,530,534,389,688,379,1001,639,1243,339,810,396,1164,1342,1143,778,804,619,452],
                        [804,981,1044,597,144,340,457,538,1029,379,945,170,863,368,898,1044,1027,423,871,291,385],
                        [1126,1843,407,1146,1201,1140,1103,1404,671,1414,655,1117,753,1168,576,407,515,913,881,1142,1064],
                        [1462,0,1520,1043,1047,902,1138,699,1514,1079,1707,852,1323,866,1617,1520,1656,1291,1317,1132,965],
                        [868,1520,0,1134,1007,1151,845,1415,818,1156,397,1060,920,1207,318,0,662,692,1028,852,1075],
                        [1184,1043,1134,0,679,566,882,704,725,784,1093,484,535,627,1014,1122,867,649,485,725,453],
                        [725,1047,1007,679,0,415,300,629,1039,416,915,261,873,475,825,1013,1037,392,979,260,387],
                        [855,902,1151,566,415,0,507,459,860,415,941,115,694,214,862,1040,857,433,819,376,227],
                        [486,1138,845,882,300,507,0,727,1045,358,731,363,879,471,641,824,1011,501,1039,371,487],
                        [1154,699,1415,704,629,459,727,0,1216,732,1366,445,1025,327,1276,1336,1154,729,1019,725,515],
                        [1284,1514,818,725,1039,860,1045,1216,0,1228,985,928,333,979,895,772,142,724,463,953,841],
                        [755,1079,1156,784,416,415,358,732,1228,0,1000,308,1062,461,910,1124,1226,769,1057,637,575],
                        [623,1707,397,1093,915,941,731,1366,985,1000,0,958,844,1064,74,527,753,612,1067,875,991],
                        [774,852,1060,484,261,115,363,445,928,308,958,0,754,246,869,1047,918,463,749,324,267],
                        [1150,1323,920,535,873,694,879,1025,333,1062,844,754,0,789,803,778,443,534,304,763,654],
                        [936,866,1207,627,475,214,471,327,979,461,1064,246,789,0,943,1121,939,514,900,458,308],
                        [548,1617,318,1014,825,862,641,1276,895,910,74,869,803,943,0,452,678,538,992,800,916],
                        [868,1520,0,1122,1013,1040,824,1336,772,1124,527,1047,778,1121,452,0,662,692,1028,852,1075],
                        [1097,1656,662,867,1037,857,1011,1154,142,1226,753,918,443,939,678,662,0,674,616,903,825],
                        [691,1291,692,649,392,433,501,729,724,769,612,463,534,514,538,692,674,0,725,213,472],
                        [1382,1317,1028,485,979,819,1039,1019,463,1057,1067,749,304,900,992,1028,616,725,0,995,749],
                        [737,1132,852,725,260,376,371,725,953,637,875,324,763,458,800,852,903,213,995,0,377],
                        [856,965,1075,453,387,227,487,515,841,575,991,267,654,308,916,1075,825,472,749,377,0]]
    # data['preference'] = [[1,0.7333333333,0.4666666667,0.7333333333,0.6,0.4666666667,0.6,0.4666666667,0.6,1,0.4,0.6,0.6,0.4666666667,0.4,0.2,0.4666666667,0.2,0.6,0.6,1],
    #                     [1,0.2,0.7333333333,0.2,0.6,0.2,0.6,0.2,0.6,0.2,0.6,0.2,0.2,0.4666666667,0.6,0.7333333333,0.2,1,0.2,0.6,0.2],
    #                     [1,0.6,0.6,1,0.6,0.6,0.6,0.6,0.6,1,0.2,1,0.6,0.2,0.6,0.2,0.6,0.2,1,0.6,1],
    #                     [1,0.2,1,0.6,0.6,0.2,0.6,0.2,0.6,0.6,0.6,0.6,0.2,0.6,1,0.6,0.2,1,0.6,1,0.6],
    #                     [1,0.7333333333,1,0.7333333333,0.6,0.4666666667,0.2,0.4666666667,0.2,1,0.7333333333,0.6,0.4666666667,0.7333333333,0.7333333333,0.7333333333,0.4666666667,1,0.6,1,1],
    #                     [1,0.2,0.7333333333,0.4666666667,0.6,0.4666666667,0.6,0.4666666667,0.6,0.2,0.4666666667,0.6,0.2,0.4666666667,0.7333333333,0.7333333333,0.4666666667,1,0.6,0.6,0.2],
    #                     [1,1,0.2,1,1,1,1,1,1,1,0.2,1,1,0.2,0.2,0.2,1,0.2,1,0.2,1],
    #                     [1,0.2,1,0.6,0.6,0.2,0.6,0.2,0.6,0.6,0.6,0.6,0.2,0.6,1,0.6,0.2,1,0.6,1,0.6],
    #                     [1,0.2,0.7333333333,0.2,0.6,0.2,0.6,0.2,0.6,0.2,0.6,0.2,0.2,0.4666666667,0.6,0.7333333333,0.2,1,0.2,0.6,0.2],
    #                     [1,1,1,1,0.2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
    #                     [1,1,0.7333333333,0.4666666667,0.6,0.4666666667,0.6,0.4666666667,0.6,0.2,0.4666666667,0.6,0.2,0.4666666667,0.7333333333,0.7333333333,0.4666666667,1,0.6,0.6,0.2],
    #                     [1,0.7333333333,1,0.7333333333,1,1,1,1,1,0.6,0.2,1,0.7333333333,0.4666666667,0.2,0.4666666667,1,0.2,1,0.2,0.6],
    #                     [1,0.4666666667,0.7333333333,1,1,0.4666666667,1,0.4666666667,1,0.2,0.7333333333,0.2,0.2,0.7333333333,0.4666666667,1,0.4666666667,1,0.2,0.6,0.2],
    #                     [1,0.6,1,1,1,0.6,0.6,0.6,0.6,1,0.6,1,0.6,0.6,1,0.6,0.6,1,1,1,1],
    #                     [1,0.4666666667,1,0.4666666667,0.6,1,0.6,0.2,0.6,0.6,0.7333333333,0.2,0.2,0.7333333333,0.7333333333,0.7333333333,0.2,1,0.2,1,0.6],
    #                     [1,0.6,1,1,0.6,0.6,1,0.6,0.2,1,0.6,1,0.6,0.6,1,0.6,0.6,1,1,1,1],
    #                     [1,0.4666666667,1,0.4666666667,0.6,0.2,0.6,1,0.6,0.6,0.7333333333,0.2,0.2,0.7333333333,0.7333333333,0.7333333333,0.2,1,0.2,1,0.6],
    #                     [1,0.6,1,1,0.6,0.6,0.2,0.6,1,1,0.6,1,0.6,0.6,1,0.6,0.6,1,1,1,1],
    #                     [1,0.2,0.6,0.2,1,0.6,1,0.6,1,1,0.6,0.6,0.2,0.6,0.6,1,0.6,1,0.6,0.6,0.2],
    #                     [1,0.4666666667,0.2,0.7333333333,0.6,0.7333333333,0.6,0.7333333333,0.6,0.6,1,1,0.6,0.2,0.4,0.2,0.7333333333,0.2,1,0.2,0.6],
    #                     [1,0.6,1,0.2,1,0.2,1,0.2,1,0.6,1,1,0.2,1,0.6,1,0.2,1,0.2,1,0.6],
    #                     [1,0.2,0.7333333333,0.2,0.6,0.2,0.6,0.2,0.6,0.2,0.6,0.2,1,0.4666666667,0.6,0.7333333333,0.2,1,0.2,0.6,0.2],
    #                     [1,0.4666666667,0.4666666667,0.7333333333,0.6,0.7333333333,0.6,0.7333333333,0.6,0.6,0.2,1,0.4666666667,1,0.4666666667,0.4666666667,0.7333333333,0.2,1,0.2,0.6],
    #                     [1,0.7333333333,0.2,0.4666666667,1,0.7333333333,1,0.7333333333,1,0.6,0.4,0.6,0.6,0.4666666667,1,0.4666666667,0.7333333333,0.2,0.6,0.2,0.6],
    #                     [1,0.7333333333,0.4666666667,1,0.6,0.7333333333,0.6,0.7333333333,0.6,1,0.2,1,0.7333333333,0.4666666667,0.4666666667,1,0.7333333333,0.2,1,0.6,1],
    #                     [1,0.4666666667,1,0.4666666667,0.6,0.2,0.6,0.2,0.6,0.6,0.7333333333,0.2,0.2,0.7333333333,0.7333333333,0.7333333333,1,1,0.2,1,0.6],
    #                     [1,1,0.2,1,1,1,1,1,1,1,0.2,1,1,0.2,0.2,0.2,1,1,1,0.2,1],
    #                     [1,0.6,1,0.2,1,0.2,1,0.2,1,0.6,1,0.2,0.2,1,0.6,1,0.2,1,1,1,0.6],
    #                     [1,0.6,0.2,0.6,1,1,1,1,1,0.6,0.2,1,0.6,0.2,0.2,0.6,1,0.2,1,1,0.6],
    #                     [1,0.2,0.6,0.2,1,0.6,1,0.6,1,0.2,0.6,0.6,0.2,0.6,0.6,1,0.6,1,0.6,0.6,1]]
    return data

In [ ]:
def create_data_model():
   """Stores the data for the problem."""
   data = {}
   data['Vs'] = [3, 5, 13, 14, 17, 26, 29, 40, 41, 44, 55, 58, 70, 80, 86, 88, 96, 97, 99, 102, 109, 111, 119, 120, 121, 133, 141, 149, 151, 153]
   data['Vc'] = [157, 160, 168, 177, 190, 196, 205, 208, 222, 229, 235, 237, 242, 245, 248, 249, 264, 274, 282, 301, 304, 308, 315, 324, 325, 333, 334, 336, 340, 357, 374, 394, 399, 410, 411, 431, 435, 455, 466, 475, 486, 504, 505, 511, 517, 539, 542, 547, 548, 550, 554, 570, 578, 584, 593, 604, 607, 621, 628, 636, 640, 646, 658, 669, 691, 698, 707, 709]
   data['Q'] = [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 7, 7, 7, 7, 7]
   data['T'] = [3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600]
   data['distance'] = [[5945,5095,8433,4113,4080,2195,3817,408,7770,4305,7509,3146,5903,1784,7159,8433,8189,4936,6153,4225,2625,3924,6743,3548,8489,3107,8560,3072,4316,1948,8429,6721,5095,3195,2693,3156,6436,2980,3454,4299,3146,3422,4726,7770,5632,2643,6055,3268,3268,3268,3268,2365,6929,3626,3229,6432,6721,4987,5872,7803,4332,4616,2427,13261,4987,6149,8433,7582,5658],
                        [6536,3977,9076,3358,4401,2318,4408,922,7015,6567,8670,3467,5148,2501,7750,9076,8432,5191,5398,4546,2880,2837,7334,3869,8740,2352,8811,3663,3561,2071,8680,7991,3977,3786,2340,4220,6686,3236,4020,4669,3467,3963,3639,7015,6172,2173,5300,2513,3175,2513,3175,2628,8199,3881,2474,7702,7991,3870,6463,8089,4702,4937,1013,13850,3870,6400,9076,6827,4903],
                        [6043,12573,1788,6739,7149,7045,6547,8618,4219,10344,3351,7244,4181,7374,3071,1788,2665,5704,5164,6714,6247,11152,3863,6617,1391,7811,810,9984,6431,7409,1834,14358,12573,10106,6620,10269,4266,5675,10974,5396,7244,6073,7799,4219,5522,6832,3755,6691,5938,6691,5938,6406,6769,5596,6641,7288,14358,12465,15363,2279,5429,6032,8189,27034,12465,2937,1788,2353,5066],
                        [3824,8453,6311,4339,1418,2930,1696,4432,7132,2075,5388,1015,5287,2220,5038,6311,7491,3496,6379,2725,2672,7031,4622,2048,5708,3327,7863,1715,4542,3293,6551,4298,8453,1837,2740,2000,4960,3027,4331,2848,1015,2142,5413,7132,2818,2691,4964,3494,3315,3494,3315,2413,3896,2543,3455,3706,4298,8345,4514,7105,2881,3116,4069,14339,8345,5452,6311,6443,5884],
                        [8022,3780,9733,4137,5058,3183,5894,2175,7672,7224,9327,4124,5805,3366,9047,9733,9089,5848,5864,5203,3537,1728,8375,4526,9397,3092,9468,5149,4347,2936,9337,9447,3780,5272,2997,5837,7343,3893,5637,5326,4124,4620,3807,7672,6829,2830,5957,3170,3832,3170,3832,3312,9006,4538,3131,8509,9447,3673,7949,8746,5359,5594,1670,15467,3673,7057,9733,7484,5369],
                        [8202,5281,10153,7553,5940,4577,5537,3749,10147,5547,10224,5356,8302,3693,9874,10153,10506,7253,9593,6303,5547,5980,9457,5626,11235,6740,10878,4077,7756,4103,10747,6856,5281,4199,5919,3085,8753,5629,0,6616,5356,5739,7833,10147,7280,5859,7978,6708,6576,6708,6576,5433,7064,5943,6669,6567,6856,5250,6006,10120,6649,6694,5208,11468,5250,8467,10153,9458,9098],
                        [7623,9725,5969,4522,5424,5320,6819,6808,1588,8619,6047,5519,1454,5649,5697,5969,3005,3979,2553,4989,3822,8304,5919,4892,5060,4890,4348,8259,4406,5684,5000,10842,9725,8381,4355,8544,4329,3405,9249,3671,5519,4348,4921,1588,5794,4340,1940,3842,3591,3842,3591,4681,8826,3871,3792,8887,10842,9617,11058,2662,3704,4898,5341,18374,9617,3000,5969,2271,3135],
                        [7209,4661,8665,2947,3990,2412,5777,1815,6604,6156,8259,3056,4737,2717,7979,8665,8021,4780,4987,4135,2469,3075,7307,3458,8329,1941,8400,4500,3150,2165,8269,8379,4661,4623,1928,4785,6275,2825,4935,4258,3056,3552,3876,6604,5761,1761,4889,2102,2764,2102,2764,2243,7977,3470,2063,7787,8379,4554,7300,7678,4291,4526,22,14765,4554,5989,8665,6416,4492],
                        [4129,7785,5773,3671,605,2262,2765,3764,6008,3339,5367,978,4163,2590,4891,5773,6367,1601,5711,1060,1392,6363,4475,383,5170,2658,6739,2979,3874,2625,6013,5562,7785,3101,2071,3264,4422,1491,5595,1712,978,1006,4744,6008,2681,2022,3840,2826,2646,2826,2646,1744,4491,1407,2787,4377,5562,7677,5778,5981,1745,1451,3401,15603,7677,4328,5773,5319,5216],
                        [6007,13056,1752,7222,7632,7528,6511,9101,4702,8677,3315,7727,4664,7857,3035,1752,3148,6187,5647,7197,6730,11635,3827,7100,2254,8294,1293,10467,6914,7892,1798,7627,13056,10589,7103,10752,4749,6158,11457,5879,7727,6556,8282,4702,5486,7315,4238,7174,6421,7174,6421,6889,6733,6079,7124,7252,7627,12948,9833,2762,5912,5996,8672,27417,12948,3420,1752,2836,5549],
                        [6043,6677,9680,6453,5067,3704,4664,3563,9273,4702,8065,4483,7428,2820,7715,9680,9632,6379,8493,5429,4674,6174,7298,4752,9076,5867,10004,3204,7032,3229,9920,4697,6677,3271,5046,927,7879,4756,2725,5743,4483,4866,8027,9273,7118,4985,7105,5608,5703,5608,5703,4559,4905,5070,5569,4408,4697,6569,3847,9246,5776,5820,4999,11158,6569,7593,9680,8584,7998],
                        [11239,2245,21206,8015,8977,7614,8574,6216,11672,8584,13261,8393,9805,6730,12911,21206,13543,10290,10055,9340,7537,4689,12494,8663,20674,7009,13915,7114,8218,7140,20607,9893,2245,7236,6997,6122,11790,8666,3037,9653,8393,8776,8145,11672,10317,6830,9957,7170,7832,7170,7832,7312,10101,8980,7131,9604,9893,2213,9043,13157,9686,9731,5670,14504,2213,11504,21206,11484,9560],
                        [3291,9221,3765,3465,3009,3686,2487,5259,5145,4653,2939,2827,3300,4015,2589,3765,5170,786,4627,1382,2888,7800,2173,2477,3162,4452,5542,5567,4019,4050,4005,5268,9221,5689,3261,5852,2414,2316,7615,1027,2827,1704,5934,5145,1462,3473,2455,3357,2589,3357,2589,3047,4373,1469,3307,5117,5268,9114,7190,4784,1060,893,4838,16740,9114,3131,3765,4122,4132],
                        [2238,9134,5039,4685,1939,3815,0,4201,7534,2766,4260,2691,5689,2931,3910,5039,7222,2758,7017,2169,2906,7472,3494,2130,4436,5002,7490,3448,5264,3340,5279,3040,9134,3570,3278,3733,3688,2966,5551,3525,2691,2460,5951,7534,2167,3384,4844,3840,3935,3840,3935,3065,2600,2861,3801,2665,3040,9026,4738,6836,3558,1832,5745,14596,9026,5144,5039,6174,6522],
                        [3824,8453,6311,4339,1418,2930,1696,4432,7132,2075,5388,1015,5287,2220,5038,6311,7491,3496,6379,2725,2672,7031,4622,2048,5708,3327,7863,1715,4542,3293,6551,4298,8453,1837,2740,2000,4960,3027,4331,2848,1015,2142,5413,7132,2818,2691,4964,3494,3315,3494,3315,2413,3896,2543,3455,3706,4298,8345,4514,7105,2881,3116,4069,14339,8345,5452,6311,6443,5884],
                        [8022,3780,9733,4137,5058,3183,5894,2175,7672,7224,9327,4124,5805,3366,9047,9733,9089,5848,5864,5203,3537,1728,8375,4526,9397,3092,9468,5149,4347,2936,9337,9447,3780,5272,2997,5837,7343,3893,5637,5326,4124,4620,3807,7672,6829,2830,5957,3170,3832,3170,3832,3312,9006,4538,3131,8509,9447,3673,7949,8746,5359,5594,1670,15467,3673,7057,9733,7484,5369],
                        [3540,8733,6027,4619,994,3210,1412,4711,7411,1864,5104,1294,5566,2283,4754,6027,7770,2440,6659,1899,2951,7311,4338,1591,5424,3606,8142,2314,4822,3573,6267,4086,8733,2436,3019,2599,4676,3306,4930,3128,1294,2422,5692,7411,2550,2970,5243,3774,3594,3774,3594,2692,3214,2823,3735,3024,4086,8625,5113,7384,3161,2289,4349,14938,8625,5731,6027,6722,6164],
                        [7721,8801,6719,2030,6033,5269,6383,5884,3330,8199,7295,5099,1716,5725,6945,6719,4747,4077,339,5087,3504,7380,7167,5501,5810,3966,5098,7839,1722,5340,5750,10422,8801,7961,3432,8124,4592,3246,9074,3769,5099,4446,2662,3330,5892,3416,2690,2919,2867,2919,2867,3898,10020,3969,2869,9830,10422,8693,10638,4404,3802,4996,4417,18322,8693,3750,6719,3021,125],
                        [5115,8696,8752,8484,5254,4583,4368,4442,10152,3774,7137,5160,8307,3699,6787,8752,10511,7258,10524,6870,5553,7053,6370,6193,8148,7472,10883,4083,8687,4108,8992,3769,8696,2343,6885,1805,7811,5635,3416,6622,5160,6287,9558,10152,6190,6836,7984,7639,6582,7639,6582,6558,3977,5949,7600,3480,3769,7448,2919,10125,6655,6154,5878,10835,7448,8472,8752,9463,10029],
                        [5836,6452,7489,1198,3684,2920,4033,3534,4856,5850,6499,2750,2988,3375,6219,7489,6273,3030,3238,2767,1154,5030,5557,3152,6580,1617,6651,5490,1401,2990,6520,8073,6452,5612,1082,5775,4526,1124,6725,2393,2750,1745,3166,4856,3688,1067,3141,215,1015,215,1015,1549,7671,1738,89,7481,8073,6344,8289,5930,2426,2735,2068,15973,6344,4240,7489,4667,2743],
                        [1860,10052,5497,6060,3059,4935,1846,5321,8910,2104,3882,4232,7065,4051,3532,5497,7795,4086,8100,3545,4281,9549,3116,3505,4893,6544,7948,3849,6639,4460,5737,2063,10052,3154,4653,4301,5064,4341,6470,4542,4232,3836,7326,8910,3542,5908,6220,5215,5310,5215,5310,5630,1397,4237,5176,1688,2063,9944,3761,7409,4575,3207,7287,13619,9944,5602,5497,7080,7605],
                        [8812,10591,6184,5505,6290,6186,7685,7674,0,9485,6760,6385,2320,6515,6410,6184,1417,4845,3304,5855,4688,9170,6632,5758,5275,5756,4433,9125,5390,6550,5215,11708,10591,9247,5221,9410,5323,4271,10115,4537,6385,5214,5905,0,6660,5206,2806,4708,4457,4708,4457,5547,9539,4737,4658,10058,11708,10483,11924,1940,4570,5764,6207,19240,10483,3994,6184,1890,3206],
                        [2992,9258,6629,7237,3226,5828,2340,5306,10030,1652,5014,3913,8185,4035,4664,6629,8927,5047,9277,4506,5570,8755,4247,3823,6025,6225,9080,3530,7440,4444,6869,1646,9258,2360,5638,3507,5687,5925,5676,5747,3913,5041,8311,10030,4067,5589,7862,6392,6213,6392,6213,5311,1853,5442,6353,1356,1646,9150,2967,8541,5780,4168,6968,12825,9150,6734,6629,8212,8782],
                        [3086,8754,4338,2688,2129,2735,2282,4308,5163,5047,3932,1947,3318,3064,3652,4338,5188,520,4645,896,1852,7332,2877,1597,3735,3627,5560,4687,2892,3099,4578,5063,8754,4809,2224,4972,2987,1365,6664,704,1947,824,4897,5163,1257,2330,2473,2441,1947,2441,1947,2096,4168,817,2391,4912,5063,8646,7486,4802,737,355,4370,15789,8646,3149,4338,4140,4150],
                        [7248,6631,8704,1782,4029,3265,5816,3898,6581,6195,8298,3095,3874,3593,8018,8704,7998,4634,3508,4174,2323,4710,7594,3497,7091,837,7831,5835,1991,3628,7530,8418,6631,5957,2926,6120,5741,2679,6904,4297,3095,3591,1666,6581,5800,1649,4191,1957,2619,1957,2619,2599,8016,3324,1918,7826,8418,6523,8634,7655,4330,4565,2420,16318,6523,5250,8704,5754,3013],
                        [9560,2887,10467,4749,5792,4214,6895,2267,8406,6905,11582,4858,6539,4099,11232,10467,9823,6582,6789,5937,4271,2384,10815,5260,10131,4165,10202,5435,4952,3967,10071,8214,2887,5557,3731,4443,8077,4627,4243,6060,4858,5354,4880,8406,7563,3564,6691,3904,4566,3904,4566,4046,8422,5272,3865,7925,8214,2779,7364,9480,6093,6328,2404,14073,2779,7791,10467,8218,6294],
                        [4008,7599,5702,3485,793,1524,2644,3578,5599,3892,5296,792,3754,1853,5016,5702,5958,1944,5525,939,1035,6177,4354,262,5099,2472,6330,3532,3688,1888,5942,6115,7599,3654,1885,3817,3603,1095,5453,1295,792,589,4558,5599,2560,1836,3431,2640,2029,2640,2029,885,4370,990,2601,4256,6115,7491,6331,5572,1328,1330,3215,16156,7491,3919,5702,4910,5030],
                        [6558,5724,8509,5682,4296,2933,3893,3221,8503,3903,8580,3712,6658,2049,8230,8509,8862,5609,7722,4659,3903,5221,7813,3982,9591,5096,9234,2433,6261,2459,9103,5212,5724,2555,4275,1441,7109,3985,1912,4972,3712,4095,7073,8503,5636,4215,6334,4837,4932,4837,4932,3789,5420,4299,4798,4923,5212,5616,4362,8476,5005,5050,4448,11743,5616,6823,8509,7814,7227],
                        [22736,14017,33282,19211,20474,19111,20071,17989,23493,20081,24758,19890,21304,18227,24408,33282,25040,21787,20938,20837,20081,15286,23991,20160,32750,18166,25412,18611,19421,18637,32683,21390,14017,18733,18769,17619,23287,20163,14534,21150,19890,20273,18880,23493,21814,18602,21621,18942,19605,18942,19605,19974,21598,20477,18903,21101,21390,13986,20540,24654,21183,21228,17443,22298,13986,23001,33282,23184,20443],
                        [5686,5888,7142,1774,2467,1703,4254,2512,5431,4633,6736,1533,3564,2031,6456,7142,6582,3383,3814,2612,1296,4466,6032,1935,6539,762,6954,4273,1977,2066,6823,6856,5888,4395,1364,4558,4829,1651,5631,2735,1533,2029,3041,5431,4238,589,3716,929,1591,929,1591,1037,6454,2020,890,6264,6856,5780,7072,6196,2768,3003,1504,14756,5780,4543,7142,5243,3319],
                        [11788,0,23024,7029,8071,6494,9123,5230,10686,9133,13810,7137,8819,7279,13460,23024,12103,8862,9069,8216,6551,3703,13043,7539,22492,6022,12482,7663,7232,6247,22425,10442,0,7785,6010,6671,10357,6907,5281,8339,7137,7633,7158,10686,9842,5843,8971,6183,6846,6183,6846,6325,10650,7552,6144,10153,10442,1717,9592,11760,8372,8607,4683,16748,1717,10071,23024,10498,8574],
                        [4727,23024,0,7096,5859,7509,5231,9082,5681,7397,2035,6422,5315,7738,1755,0,4127,4609,6626,5206,6711,11509,2547,6072,1320,8275,3172,7777,6788,7873,577,6347,12930,7946,7084,8062,3184,6139,10358,4851,6422,5299,8505,5681,4206,7296,4546,7048,6413,7048,6413,6870,5453,5292,6998,5972,6347,12822,8553,3741,4884,4716,8546,18411,12822,3260,0,3815,6528],
                        [5698,7029,7096,0,4224,3460,4573,4074,4875,6390,6526,3290,2686,3915,6246,7311,6292,3075,2320,3007,1694,5570,5602,3692,5903,2157,6643,6030,803,3530,6342,8613,6991,6152,1622,6315,4553,1673,7265,2439,3290,2326,2520,4875,3869,1606,3003,1109,1196,1109,1196,2088,8211,1919,1059,8021,8613,6884,8829,5949,2472,2916,2608,16513,6884,4062,7311,4566,1825],
                        [4067,8071,5859,4224,0,2236,1939,4344,6311,2994,4747,1558,4466,3334,4397,5706,6670,1534,5501,993,1682,6943,3981,685,5103,3238,7042,3136,4040,2600,5946,5216,8365,3258,2054,3421,4355,1742,5954,1942,1558,1236,4727,6311,2187,2023,4143,2616,2741,2616,2741,1597,3741,1637,2577,3551,5216,8257,5624,6284,1975,1383,3981,15079,8257,4631,5706,5622,5006],
                        [4988,6494,7509,3460,2236,0,3185,2377,5738,4148,6425,1048,3893,1154,6145,6831,6097,2844,4957,1894,1138,6035,5371,1217,6398,2331,6469,3788,3496,1189,6338,6371,7457,3910,1510,4073,4344,1221,4754,2208,1048,1330,4183,5738,3540,1450,3569,2072,2168,2072,2168,1024,5969,1535,2033,5779,6371,7349,6587,5711,2241,2285,3073,13880,7349,4058,6831,5049,4462],
                        [2238,9123,5231,4573,1939,3185,0,4201,7534,2766,4260,2691,5689,2931,3910,5039,7222,2758,7017,2169,2906,7472,3494,2130,4436,5002,7490,3448,5264,3340,5279,3040,9134,3570,3278,3733,3688,2966,5551,3525,2691,2460,5951,7534,2167,3384,4844,3840,3935,3840,3935,3065,2600,2861,3801,2665,3040,9026,4738,6836,3558,1832,5745,14596,9026,5144,5039,6174,6522],
                        [6199,5230,9082,4074,4344,2377,4201,0,7634,4559,7763,3010,5767,1986,7413,8499,7765,4512,6017,4089,2489,3670,6997,3412,8066,2971,8137,3326,4180,1588,8006,6975,4810,3449,2557,3611,6012,2844,3819,3875,3010,3286,4472,7634,5496,2507,5919,3132,3132,3132,3132,2229,7183,3202,3093,6686,6975,4703,6126,7379,3908,4480,1846,13649,4703,5726,8499,7446,5522],
                        [8812,10686,5681,4875,6311,5738,7534,7634,0,9485,6760,6385,2320,6515,6410,6184,1417,4845,3304,5855,4688,9170,6632,5758,5275,5756,4433,9125,5390,6550,5215,11708,10591,9247,5221,9410,5323,4271,10115,4537,6385,5214,5905,0,6660,5206,2806,4708,4457,4708,4457,5547,9539,4737,4658,10058,11708,10483,11924,1940,4570,5764,6207,19240,10483,3994,6184,1890,3206],
                        [4177,9133,7397,6390,2994,4148,2766,4559,9485,0,6199,3093,7365,3429,5849,7205,9569,4440,8457,3899,4750,9109,5432,4126,6602,5405,9941,2924,6620,3838,7445,2278,9123,3046,4818,3209,5854,5105,5540,4926,3093,4220,7491,9210,4333,4769,7042,5572,5393,5572,5393,4491,3038,4621,5533,2541,2278,9015,4152,9183,4959,3998,6147,14010,9015,7530,7205,8521,7962],
                        [3375,13810,2035,6526,4747,6425,4260,7763,6760,6199,0,6789,5050,7086,350,2890,5189,3872,6753,5958,6694,10772,1195,5918,2287,8469,5770,7289,6051,7136,3130,4995,13492,6594,7066,7741,2447,5402,9910,4114,6789,4562,7768,6743,3094,7172,4281,6311,5676,6311,5676,6133,4101,4555,6261,4620,4995,13384,7201,4803,4147,3979,7809,17144,13384,2995,2890,4474,6655],
                        [4842,7137,6422,3290,1558,1048,2691,3010,6385,3093,6789,0,4264,2236,5547,6233,6468,2474,5357,1703,1649,6009,5640,1026,5630,2304,6840,2733,3520,2271,6709,5316,7431,2855,1717,3018,4715,2004,5349,1826,0,1120,4390,6109,3329,1668,3941,2472,2292,2472,2292,1390,4914,1521,2433,4724,5316,7323,5532,6082,1859,2094,3047,15357,7323,4429,6233,5420,4862],
                        [6595,8819,5315,2686,4466,3893,5689,5767,2320,7365,5050,4264,0,4621,4669,5664,3592,2951,1699,3961,3353,7230,4891,3864,4755,3816,4043,7689,2357,5190,4695,10272,8651,7811,3281,7974,3301,2377,8221,2643,4949,3320,4333,2174,4766,3266,913,2768,2519,2768,2519,3748,7798,2843,2718,8317,10272,8543,10488,3248,2676,3870,4267,17346,8543,1972,5664,1966,1600],
                        [5760,7279,7738,3915,3334,1154,2931,1986,6515,3429,7086,2236,4621,0,6916,7602,6868,3615,5729,2665,1910,5197,6142,1988,7169,3103,7240,2713,4268,465,7109,7143,6618,2835,2282,2998,5115,1992,4044,2979,1820,2102,4955,6509,4312,2221,4341,2844,2939,2844,2939,1795,6570,2306,2805,6073,7143,6511,5512,6482,3012,3056,2235,13169,6511,4829,7602,5820,5234],
                        [3025,13460,1755,6246,4397,6145,3910,7413,6410,5849,350,5547,4669,6916,0,2540,4839,3522,6403,5608,6344,10422,845,5568,1937,8119,4991,6939,5701,6786,2780,4645,13142,6244,6716,7391,2097,5052,9560,3764,6439,4212,7418,6393,2744,6822,3931,5961,5326,5961,5326,5783,3751,4205,5911,4270,4645,13034,6851,4453,3797,3629,7459,16709,13034,2645,2540,4124,6305],
                        [4727,23024,0,7311,5706,6831,5039,8499,6184,7205,2890,6233,5664,7602,2540,0,4127,4609,6626,5206,6711,11509,2547,6072,1320,8275,3172,7777,6788,7873,577,6347,12930,7946,7084,8062,3184,6139,10358,4851,6422,5299,8505,5681,4206,7296,4546,7048,6413,7048,6413,6870,5453,5292,6998,5972,6347,12822,8553,3741,4884,4716,8546,18411,12822,3260,0,3815,6528],
                        [7118,12103,4127,6292,6670,6097,7222,7765,1417,9569,5189,6468,3592,6868,4839,4127,0,5067,4721,6077,5610,10587,4938,5980,3581,7174,2823,9347,6807,6772,3521,8739,12008,9469,5983,9632,3629,5038,10337,4759,6607,5436,7322,1417,6284,6195,3118,6125,5301,6125,5301,5769,7845,4959,6075,8364,8739,11900,12146,587,4792,5986,7624,19462,11900,2300,4490,1407,4623],
                        [3956,8862,4609,3075,1534,2844,2758,4512,4845,4440,3872,2474,2951,3615,3522,4609,5067,0,4394,863,1966,7446,2839,1373,3696,3741,5309,4801,3786,3213,4539,5933,8868,4923,2338,5086,2948,1479,6778,794,2061,938,5011,4912,2127,2444,2222,2555,2356,2555,2356,2210,5038,930,2505,5782,5933,8760,7600,4551,308,772,4484,15903,8760,2898,4299,3889,3899],
                        [8094,9069,6626,2320,5501,4957,7017,6017,3304,8457,6753,5357,1699,5729,6403,6626,4721,4394,0,5460,3760,7636,7141,5757,5784,4222,5072,8095,1978,5596,5724,10678,9057,8217,3688,8380,4965,3619,9330,4142,5355,4819,2937,3304,6265,3672,2664,3175,3240,3175,3240,4154,10048,4342,3125,10086,10678,8949,10894,4378,4175,5369,4673,18578,8949,3724,6693,2995,464],
                        [3783,8216,5206,3007,993,1894,2169,4089,5855,3899,5958,1703,3961,2665,5608,5206,6077,863,5460,0,1608,7088,3574,831,4432,3383,5903,4443,3966,2770,5275,6120,8510,4565,1980,4728,3684,1258,6335,1047,1703,506,4653,5506,1954,2086,2816,2542,2290,2542,2290,1767,4371,1159,2503,4456,6120,8402,7242,5145,1080,713,4126,15460,8402,3492,5035,4483,4493],
                        [4692,6551,6711,1694,1682,1138,2906,2489,4688,4750,6694,1649,3353,1910,6344,6711,5610,1966,3760,1608,0,5134,4907,1207,5934,1721,6004,4544,2581,1969,5874,7127,6555,4666,595,4829,3880,416,5534,1744,1804,1034,3268,4688,3244,701,2972,874,959,874,959,561,5054,1071,835,5202,7127,6448,7343,5247,1777,2014,2172,14659,6448,3593,6367,4379,3547],
                        [11320,3703,11509,5570,6943,6035,7472,3670,9170,9109,10772,6009,7230,5197,10422,11509,10587,7446,7636,7088,5134,0,12575,6118,10989,4683,11060,7196,5938,4527,10929,9974,3712,7318,4589,6204,8935,5485,6004,6918,5716,6212,5398,9264,8421,4422,7549,4762,5424,4762,5424,4904,10182,6130,4723,9685,9974,3604,9125,10338,6951,7186,3262,15834,3604,8649,11325,9076,6960],
                        [2610,13043,2547,5602,3981,5371,3494,6997,6632,5432,1195,5640,4891,6142,845,2547,4938,2839,7141,3574,4907,12575,0,5154,2023,7705,5078,6525,6073,6223,2867,4231,12728,5830,6302,6977,2468,4489,9146,3201,6025,3878,8975,6479,2330,6408,4018,6333,4763,6333,4763,5220,3337,3642,6283,3856,4231,12620,6437,4539,3234,3066,8448,16295,12620,2732,2627,4210,6392],
                        [3900,7539,6072,3692,685,1217,2130,3412,5758,4126,5918,1026,3864,1988,5568,6072,5980,1373,5757,831,1207,6118,5154,0,4941,2706,6510,3766,3399,2068,5784,5813,7833,3888,1413,4051,4193,1101,5633,1302,1026,596,4086,5779,2452,1491,3611,1975,2209,1975,2209,1066,4262,997,1936,4148,5813,7725,6565,5752,1335,1222,3449,16390,7725,4099,5544,5090,4365],
                        [4381,22492,1320,5903,5103,6398,4436,8066,5275,6602,2287,5630,4755,7169,1937,1320,3581,3696,5784,4432,5934,10989,2023,4941,0,7149,2484,6980,5574,6747,944,6002,11911,7601,5958,8748,2386,5013,20512,4053,6582,4502,7291,5280,3409,6170,3092,6029,5276,6029,5276,5744,5108,4495,5979,5627,6002,11803,8208,3340,4086,3919,7527,26942,11803,1806,1543,3011,5723],
                        [6458,6022,8275,2157,3238,2331,5002,2971,5756,5405,8469,2304,3816,3103,8119,8275,7174,3741,4222,3383,1721,4683,7705,2706,7149,0,7726,5045,1564,2838,7595,7628,6013,5167,2135,5330,5601,2422,6403,3507,2305,2801,2610,5859,5010,1017,4144,1357,2019,1357,2019,1808,7226,2792,1318,7036,7628,5905,7844,6968,3540,3775,1629,15528,5905,5315,7914,5671,2586],
                        [7422,12482,3172,6643,7042,6469,7490,8137,4433,9941,5770,6840,4043,7240,4991,3172,2823,5309,5072,5903,6004,11060,5078,6510,2484,7726,0,9892,6339,7317,2453,9043,12481,10014,6528,10177,4174,5583,10882,5304,7152,5981,7707,4127,6829,6740,3663,6599,5846,6599,5846,6314,8149,5504,6549,8668,9043,12373,12691,2187,5337,6531,8097,28048,12373,2845,2407,2261,4974],
                        [5265,7663,7777,6030,3136,3788,3448,3326,9125,2924,7289,2733,7689,2713,6939,7777,9347,4801,8095,4443,4544,7196,6525,3766,6980,5045,9892,0,6260,2605,7804,5147,7653,2488,4458,1739,6213,4132,4070,4566,2733,3860,7131,8649,4692,4409,6481,5212,5033,5212,5033,4131,4531,4446,5173,4341,5147,7545,5165,8622,4599,4834,5787,13195,7545,6969,7564,7960,7602],
                        [5927,7232,6788,803,4040,3496,5264,4180,5390,6620,6051,3520,2357,4268,5701,6788,6807,3786,1978,3966,2581,5938,6073,3399,5574,1564,6339,6260,0,3759,6003,8842,7220,6381,1851,6544,4214,1902,7494,2668,3519,2555,1845,5347,4098,1835,2664,1338,1424,1338,1424,2317,8440,2148,1288,8250,8842,7113,9058,6421,2701,3145,2837,16742,7113,3723,6972,4227,1486],
                        [5478,6247,7873,3530,2600,1189,3340,1588,6550,3838,7136,2271,5190,465,6786,7873,6772,3213,5596,2770,1969,4527,6223,2068,6747,2838,7317,2605,3759,0,7209,6254,6237,2728,2120,2890,5215,2092,4086,3079,1919,2202,4807,6609,4412,2070,4440,2695,2695,2695,2695,1792,6462,2406,2656,5965,6254,6130,5405,6582,3112,3156,1854,13211,6130,4929,7702,5920,5085],
                        [4966,22425,577,6342,5946,6338,5279,8006,5215,7445,3130,6709,4695,7109,2780,577,3521,4539,5724,5275,5874,10929,2867,5784,944,7595,2453,7804,6003,7209,0,6586,13169,8185,7323,8301,3423,6378,10597,5090,6661,5538,8744,5862,4445,7535,4785,7287,6652,7287,6652,7109,5692,5531,7237,6211,6586,13061,8792,3922,5123,4955,8785,18650,13061,3499,711,3996,6709],
                        [2963,10442,6347,8613,5216,6371,3040,6975,11708,2278,4995,5316,10272,7143,4645,6347,8739,5933,10678,6120,7127,9974,4231,5813,6002,7628,9043,5147,8842,6254,6586,0,10438,3540,7041,4687,5658,7328,6856,7149,5316,6443,9714,11433,4038,6992,7991,7795,7616,7795,7616,6714,1824,6844,7756,778,0,10330,4147,8512,7182,4889,8370,14005,10330,6705,6600,8183,10185],
                        [11788,0,12930,6991,8365,7457,9134,4810,10591,9123,13492,7431,8651,6618,13142,12930,12008,8868,9057,8510,6555,3712,12728,7833,11911,6013,12481,7653,7220,6237,13169,10438,0,7785,6010,6671,10357,6907,5281,8339,7137,7633,7158,10686,9842,5843,8971,6183,6846,6183,6846,6325,10650,7552,6144,10153,10442,1717,9592,11760,8372,8607,4683,16748,1717,10071,23024,10498,8574],
                        [4910,7785,7946,6152,3258,3910,3570,3449,9247,3046,6594,2855,7811,2835,6244,7946,9469,4923,8217,4565,4666,7318,5830,3888,7601,5167,10014,2488,6381,2728,8185,3540,7785,0,4580,1861,6335,4255,4192,4688,2855,3982,7253,8772,4814,4531,6604,5334,5155,5334,5155,4253,3772,4569,5295,3275,3564,7667,2715,8745,4721,4956,5909,12540,7667,7092,8547,8083,7724],
                        [5333,6010,7084,1622,2054,1510,3278,2557,5221,4818,7066,1717,3281,2282,6716,7084,5983,2338,3688,1980,595,4589,6302,1413,5958,2135,6528,4458,1851,2120,7323,7041,6010,4580,0,5284,4735,1332,6233,2602,2259,2755,2674,5064,3885,106,3349,562,1224,562,1224,625,7180,1947,523,6990,7582,5853,7798,6138,2635,3729,1577,15482,5853,4449,7698,4876,2952],
                        [6053,6671,8062,6315,3421,4073,3733,3611,9410,3209,7741,3018,7974,2998,7391,8062,9632,5086,8380,4728,4829,6204,6977,4051,8748,5330,10177,1739,6544,2890,8301,4687,6671,1861,5284,0,6498,4417,3087,4851,3018,4145,7416,8934,4977,4694,6766,5497,5318,5497,5318,4416,4915,4731,5458,4418,4707,6561,3857,8907,4884,5119,6072,12041,6561,7254,7849,8245,7887],
                        [3636,10357,3184,4553,4355,4344,3688,6012,5323,5854,2447,4715,3301,5115,2097,3184,3629,2948,4965,3684,3880,8935,2468,4193,2386,5601,4174,6213,4214,5215,3423,5658,10357,6335,4735,6498,0,3312,8611,3299,4881,3748,5785,5353,2655,4469,1810,4328,3576,4328,3576,4043,5101,3740,4278,5620,5995,10102,8383,3413,3332,3165,5826,17736,10102,1760,3004,2751,4181],
                        [4432,6907,6139,1673,1742,1221,2966,2844,4271,5105,5402,2004,2377,1992,5052,6139,5038,1479,3619,1258,416,5485,4489,1101,5013,2422,5583,4132,1902,2092,6378,7328,6907,4255,1332,4417,3312,0,5619,1326,1889,647,3623,4271,2603,1056,2555,1075,900,1075,900,917,5117,565,1025,5257,7212,6803,7428,4829,1359,1649,2527,14744,6803,3175,5949,3962,3258],
                        [8202,5281,10358,7265,5954,4754,5551,3819,10115,5540,9910,5349,8221,4044,9560,10358,10337,6778,9330,6335,5534,6004,9146,5633,20512,6403,10882,4070,7494,4086,10597,6856,5281,4192,6233,3087,8611,5619,0,6616,5356,5739,7833,10147,7280,5859,7978,6708,6576,6708,6576,5433,7064,5943,6669,6567,6856,5250,6006,10120,6649,6694,5208,11468,5250,8467,10153,9458,9098],
                        [4318,8339,4851,2439,1942,2208,3525,3875,4537,4926,4114,1826,2643,2979,3764,4851,4759,794,4142,1047,1744,6918,3201,1302,4053,3507,5304,4566,2668,3079,5090,7149,8339,4688,2602,4851,3299,1326,6616,0,1826,703,4943,4485,2489,2482,1794,2356,1588,2356,1588,2056,5400,696,2306,5470,7149,8525,7365,4123,33,1253,4249,15749,8525,2470,4661,3461,3472],
                        [4842,7137,6422,3290,1558,1048,2691,3010,6385,3093,6789,0,4949,1820,6439,6422,6607,2061,5355,1703,1804,5716,6025,1026,6582,2305,7152,2733,3519,1919,6661,5316,7137,2855,2259,3018,4881,1889,5356,1826,0,1120,4390,6109,3329,1668,3941,2472,2292,2472,2292,1390,4914,1521,2433,4724,5316,7323,5532,6082,1859,2094,3047,15357,7323,4429,6233,5420,4862],
                        [3858,7633,5299,2326,1236,1330,2460,3286,5214,4220,4562,1120,3320,2102,4212,5299,5436,938,4819,506,1034,6212,3878,596,4502,2801,5981,3860,2555,2202,5538,6443,7633,3982,2755,4145,3748,647,5739,703,1120,0,4070,5161,2029,1503,2471,1959,1649,1959,1649,1184,4617,398,1920,4764,6443,7819,6659,4800,736,1076,3543,14877,7819,3147,5110,4138,4148],
                        [7738,7158,8505,2520,4727,4183,5951,4472,5905,7491,7768,4390,4333,4955,7418,8505,7322,5011,2937,4653,3268,5398,8975,4086,7291,2610,7707,7131,1845,4807,8744,9714,7158,7253,2674,7416,5785,3623,7833,4943,4390,4070,0,5904,6290,2781,4381,2916,3141,2916,3141,3296,9310,4284,2877,9120,9712,7207,9928,6978,4990,5060,3893,17612,7207,5440,9328,5630,2889],
                        [8812,10686,5681,4875,6311,5738,7534,7634,0,9210,6743,6109,2174,6509,6393,5681,1417,4912,3304,5506,4688,9264,6479,5779,5280,5859,4127,8649,5347,6609,5862,11433,10686,8772,5064,8934,5353,4271,10147,4485,6109,5161,5904,0,6660,5206,2806,4708,4457,4708,4457,5547,9539,4737,4658,10058,11708,10483,11924,1940,4570,5764,6207,19240,10483,3994,6184,1890,3206],
                        [2821,9842,4206,3869,2187,3540,2167,5496,6660,4333,3094,3329,4766,4312,2744,4206,6284,2127,6265,1954,3244,8421,2330,2452,3409,5010,6829,4692,4098,4412,4445,4038,9842,4814,3885,4977,2655,2603,7280,2489,3329,2029,6290,6660,0,3611,5564,4067,4162,4067,4162,3292,3183,3089,4028,3647,4022,9619,5720,6665,3427,2060,5651,15578,9619,4973,4868,6003,6457],
                        [5157,5843,7296,1606,2023,1450,3384,2507,5206,4769,7172,1668,3266,2221,6822,7296,6195,2444,3672,2086,701,4422,6408,1491,6170,1017,6740,4409,1835,2070,7535,6992,5843,4531,106,4694,4469,1056,5859,2482,1668,1503,2781,5206,3611,0,3421,634,996,634,996,519,6797,1703,595,6607,7199,5798,7415,5880,2409,2479,1522,15099,5798,4226,7000,4948,3024],
                        [5565,8971,4546,3003,4143,3569,4844,5919,2806,7042,4281,3941,913,4341,3931,4546,3118,2222,2664,2816,2972,7549,4018,3611,3092,4144,3663,6481,2664,4440,4785,7991,8971,6604,3349,6766,1810,2555,7978,1794,3941,2471,4381,2806,5564,3421,0,3085,2835,3085,2835,3411,7078,2601,3035,7597,7972,8860,9788,2906,2434,3628,4584,17104,8860,1251,4501,1587,2566],
                        [5525,6183,7048,1109,2616,2072,3840,3132,4708,5572,6311,2472,2768,2844,5961,7048,6125,2555,3175,2542,874,4762,6333,1975,6029,1357,6599,5212,1338,2695,7287,7795,6183,5334,562,5497,4328,1075,6708,2356,2472,1959,2916,4708,4067,634,3085,0,1565,0,1565,1160,7283,2071,225,7093,7685,5956,7901,6655,2777,2847,1680,15585,5956,4768,7368,5272,2531],
                        [5081,6846,6413,1196,2741,2168,3935,3132,4457,5393,5676,2292,2519,2939,5326,6413,5301,2356,3240,2290,959,5424,4763,2209,5276,2019,5846,5033,1424,2695,6652,7616,6846,5155,1224,5318,3576,900,6576,1588,2292,1649,3141,4457,4162,996,2835,1565,0,768,0,1747,7869,1302,718,7679,8271,6542,8487,5526,1872,2299,2266,16171,6542,3837,7086,4264,2340],
                        [5525,6183,7048,1109,2616,2072,3840,3132,4708,5572,6311,2472,2768,2844,5961,7048,6125,2555,3175,2542,874,4762,6333,1975,6029,1357,6599,5212,1338,2695,7287,7795,6183,5334,562,5497,4328,1075,6708,2356,2472,1959,2916,4708,4067,634,3085,0,768,0,1565,1160,7283,2071,225,7093,7685,5956,7901,6655,2777,2847,1680,15585,5956,4768,7368,5272,2531],
                        [5081,6846,6413,1196,2741,2168,3935,3132,4457,5393,5676,2292,2519,2939,5326,6413,5301,2356,3240,2290,959,5424,4763,2209,5276,2019,5846,5033,1424,2695,6652,7616,6846,5155,1224,5318,3576,900,6576,1588,2292,1649,3141,4457,4162,996,2835,1565,0,1565,0,1747,7869,1302,718,7679,8271,6542,8487,5526,1872,2299,2266,16171,6542,3837,7086,4264,2340],
                        [4708,6325,6870,2088,1597,1024,3065,2229,5547,4491,6133,1390,3748,1795,5783,6870,5769,2210,4154,1767,561,4904,5220,1066,5744,1808,6314,4131,2317,1792,7109,6714,6325,4253,625,4416,4043,917,5433,2056,1390,1184,3296,5547,3292,519,3411,1160,1747,1160,1747,0,5070,1254,1044,5218,7143,7405,7359,5431,1960,2030,3129,14675,7405,3777,6551,4768,3474],
                        [2068,10650,5453,8211,3741,5969,2600,7183,9539,3038,4101,4914,7798,6570,3751,5453,7845,5038,10048,4371,5054,10182,3337,4262,5108,7226,8149,4531,8440,6462,5692,1824,10650,3772,7180,4915,5101,5117,7064,5400,4914,4617,9310,9539,3183,6797,7078,7283,7869,7283,7869,5070,0,5023,5963,1449,1824,10538,4355,7617,5361,3994,7969,14213,10538,5810,5705,7288,8392],
                        [3851,7552,5292,1919,1637,1535,2861,3202,4737,4621,4555,1521,2843,2306,4205,5292,4959,930,4342,1159,1071,6130,3642,997,4495,2792,5504,4446,2148,2406,5531,6844,7552,4569,1947,4731,3740,565,5943,696,1521,398,4284,4737,3089,1703,2601,2071,1302,2071,1302,1254,5023,0,1647,5287,6965,7363,7181,4718,768,1068,3087,14989,7363,3065,5103,4056,3732],
                        [5811,6144,6998,1059,2577,2033,3801,3093,4658,5533,6261,2433,2718,2805,5911,6998,6075,2505,3125,2503,835,4723,6283,1936,5979,1318,6549,5173,1288,2656,7237,7756,6144,5295,523,5458,4278,1025,6669,2306,2433,1920,2877,4658,4028,595,3035,225,718,225,718,1044,5963,1647,0,7456,8048,6319,8264,5905,2401,2710,2043,15948,6319,4215,7464,4642,2718],
                        [2587,10153,5972,8021,3551,5779,2665,6686,10058,2541,4620,4724,8317,6073,4270,5972,8364,5782,10086,4456,5202,9685,3856,4148,5627,7036,8668,4341,8250,5965,6211,778,10153,3275,6990,4418,5620,5257,6567,5470,4724,4764,9120,10058,3647,6607,7597,7093,7679,7093,7679,5218,1449,5287,7456,0,771,10041,3858,8137,5881,4514,7779,13716,10041,6330,6225,7808,9593],
                        [2963,10442,6347,8613,5216,6371,3040,6975,11708,2278,4995,5316,10272,7143,4645,6347,8739,5933,10678,6120,7127,9974,4231,5813,6002,7628,9043,5147,8842,6254,6586,0,10442,3564,7582,4707,5995,7212,6856,7149,5316,6443,9712,11708,4022,7199,7972,7685,8271,7685,8271,7143,1824,6965,8048,771,0,10330,4147,8512,7182,4889,8370,14005,10330,6705,6600,8183,10185],
                        [11680,1717,12822,6884,8257,7349,9026,4703,10483,9015,13384,7323,8543,6511,13034,12822,11900,8760,8949,8402,6448,3604,12620,7725,11803,5905,12373,7545,7113,6130,13061,10330,1717,7667,5853,6561,10102,6803,5250,8525,7323,7819,7207,10483,9619,5798,8860,5956,6542,5956,6542,7405,10538,7363,6319,10041,10330,0,9484,11652,8265,8500,4576,16194,0,9963,12639,10390,8466],
                        [5472,9592,8553,8829,5624,6587,4738,6126,11924,4152,7201,5532,10488,5512,6851,8553,12146,7600,10894,7242,7343,9125,6437,6565,8208,7844,12691,5165,9058,5405,8792,4147,9592,2715,7798,3857,8383,7428,6006,7365,5532,6659,9928,11924,5720,7415,9788,7901,8487,7901,8487,7359,4355,7181,8264,3858,4147,9484,0,11422,7398,7633,8586,10207,9504,9769,9109,10760,10401],
                        [6872,11760,3741,5949,6284,5711,6836,7379,1940,9183,4803,6082,3248,6482,4453,3741,587,4551,4378,5145,5247,10338,4539,5752,3340,6968,2187,8622,6421,6582,3922,8512,11760,8745,6138,8907,3413,4829,10120,4123,6082,4800,6978,1940,6665,5880,2906,6655,5526,6655,5526,5431,7617,4718,5905,8137,8512,11652,11422,0,4546,5740,7281,19216,11557,2054,4244,820,4280],
                        [4351,8372,4884,2472,1975,2241,3558,3908,4570,4959,4147,1859,2676,3012,3797,4884,4792,308,4175,1080,1777,6951,3234,1335,4086,3540,5337,4599,2701,3112,5123,7182,8372,4721,2635,4884,3332,1359,6649,33,1859,736,4990,4570,3427,2409,2434,2777,1872,2777,1872,1960,5361,768,2401,5881,7182,8265,7398,4546,0,1286,4282,15782,8558,2503,4694,3494,3505],
                        [3688,8607,4716,2916,1383,2285,1832,4480,5764,3998,3979,2094,3870,3056,3629,4716,5986,772,5369,713,2014,7186,3066,1222,3919,3775,6531,4834,3145,3156,4955,4889,8607,4956,3729,5119,3165,1649,6694,1253,2094,1076,5060,5764,2060,2479,3628,2847,2299,2847,2299,2030,3994,1068,2710,4514,4889,8500,7633,5740,1286,0,4400,15734,8676,3397,4940,4388,4398],
                        [7231,4683,8546,2608,3981,3073,5745,1846,6207,6147,7809,3047,4267,2235,7459,8546,7624,4484,4673,4126,2172,3262,8448,3449,7527,1629,8097,5787,2837,1854,8785,8370,4683,5909,1577,6072,5826,2527,5208,4249,3047,3543,3893,6207,5651,1522,4584,1680,2266,1680,2266,3129,7969,3087,2043,7779,8370,4576,8586,7281,4282,4400,0,14787,4576,6011,8687,6438,4514],
                        [15348,16748,18411,16513,15079,13880,14596,13649,19240,14010,17144,15357,17346,13169,16709,18411,19462,15903,18578,15460,14659,15834,16295,16390,26942,15528,28048,13195,16742,13211,18650,14005,16748,12540,15482,12041,17736,14744,11468,15749,15357,14877,17612,19240,15578,15099,17104,15585,16171,15585,16171,14675,14213,14989,15948,13716,14005,16194,10207,19216,15782,15734,14787,0,16717,17590,18985,18581,17995],
                        [11680,1717,12822,6884,8257,7349,9026,4703,10483,9015,13384,7323,8543,6511,13034,12822,11900,8760,8949,8402,6448,3604,12620,7725,11803,5905,12373,7545,7113,6130,13061,10330,1717,7667,5853,6561,10102,6803,5250,8525,7323,7819,7207,10483,9619,5798,8860,5956,6542,5956,6542,7405,10538,7363,6319,10041,10330,0,9504,11557,8558,8676,4576,16717,0,9963,12639,10390,8466],
                        [5064,10071,3260,4062,4631,4058,5144,5726,3994,7530,2995,4429,1972,4829,2645,3260,2300,2898,3724,3492,3593,8649,2732,4099,1806,5315,2845,6969,3723,4929,3499,6705,10071,7092,4449,7254,1760,3175,8467,2470,4429,3147,5440,3994,4973,4226,1251,4768,3837,4768,3837,3777,5810,3065,4215,6330,6705,9963,9769,2054,2503,3397,6011,17590,9963,0,3667,1409,3626],
                        [4727,23024,0,7311,5706,6831,5039,8499,6184,7205,2890,6233,5664,7602,2540,0,4490,4299,6693,5035,6367,11325,2627,5544,1543,7914,2407,7564,6972,7702,711,6600,23024,8547,7698,7849,3004,5949,10153,4661,6233,5110,9328,6184,4868,7000,4501,7368,7086,7368,7086,6551,5705,5103,7464,6225,6600,12639,9109,4244,4694,4940,8687,18985,12639,3667,0,3815,6528],
                        [6509,10498,3815,4566,5622,5049,6174,7446,1890,8521,4474,5420,1966,5820,4124,3815,1407,3889,2995,4483,4379,9076,4210,5090,3011,5671,2261,7960,4227,5920,3996,8183,10498,8083,4876,8245,2751,3962,9458,3461,5420,4138,5630,1890,6003,4948,1587,5272,4264,5272,4264,4768,7288,4056,4642,7808,8183,10390,10760,820,3494,4388,6438,18581,10390,1409,3815,0,2896],
                        [7482,8574,6528,1825,5006,4462,6522,5522,3206,7962,6655,4862,1600,5234,6305,6528,4623,3899,464,4493,3547,6960,6392,4365,5723,2586,4974,7602,1486,5085,6709,10185,8574,7724,2952,7887,4181,3258,9098,3472,4862,4148,2889,3206,6457,3024,2566,2531,2340,2531,2340,3474,8392,3732,2718,9593,10185,8466,10401,4280,3505,4398,4514,17995,8466,3626,6528,2896,0]]
   data['duration'] = [[1027,707,1332,686,582,374,602,132,1157,605,1239,387,966,315,1149,1332,1208,784,960,667,457,542,1082,519,1273,426,1302,460,708,313,1265,805,707,432,411,462,971,505,450,714,387,596,683,1157,892,397,960,498,614,498,614,354,909,630,492,853,805,684,700,1159,720,743,382,1394,684,982,1332,1159,858],
                    [1130,539,1349,537,541,362,705,216,1008,646,1250,346,817,349,1252,1349,1150,784,811,626,458,316,1185,478,1270,277,1299,563,559,304,1262,896,539,535,360,537,967,506,477,673,346,527,457,1008,836,282,811,350,492,350,492,366,1000,630,344,944,896,516,803,1102,679,702,171,1412,516,979,1349,1010,709],
                    [1113,1742,394,1045,1100,1039,1090,1303,570,1316,642,1016,652,1067,563,394,414,812,780,1041,963,1478,763,1037,294,1061,233,1293,966,1069,377,1187,1742,1266,999,1332,602,855,1455,778,1016,899,1168,570,912,1061,555,967,912,967,912,993,1173,840,967,1281,1187,1719,1310,365,784,997,1176,1949,1719,452,394,369,712],
                    [701,934,1006,550,184,292,276,491,995,226,913,74,829,289,823,1006,993,538,824,399,341,670,756,251,912,252,1087,203,572,322,1032,442,934,176,295,242,705,389,490,446,74,300,602,995,474,281,694,363,498,363,498,238,595,372,357,558,442,911,444,944,452,475,368,1321,911,767,1006,932,722],
                    [1288,565,1467,632,659,499,863,406,1125,764,1368,464,934,486,1289,1467,1267,902,916,744,576,195,1320,596,1387,337,1416,721,630,441,1379,980,565,693,478,693,1084,624,633,791,464,645,506,1125,954,400,928,467,609,467,609,495,1170,748,461,1114,980,542,961,1219,797,820,288,1568,542,1096,1467,1127,814],
                    [1049,496,1447,1011,746,531,717,513,1346,658,1294,571,1180,445,1204,1447,1343,919,1285,862,712,565,1137,714,1402,714,1437,486,1033,502,1400,693,496,458,748,334,1106,727,0,849,571,747,932,1346,996,745,1044,824,912,824,912,678,797,765,818,741,693,502,594,1294,855,938,645,1053,502,1117,1447,1282,1183],
                    [1311,1433,839,681,983,922,1127,1113,247,1199,1059,899,304,950,969,839,388,695,379,924,812,1169,1008,920,754,800,705,1176,537,952,746,1415,1433,1149,792,1215,715,657,1338,661,899,782,668,247,949,783,373,658,636,658,636,876,1418,722,658,1525,1415,1410,1417,341,667,907,867,2204,1410,553,839,379,365],
                    [1126,660,1342,530,534,389,688,379,1001,639,1243,339,810,396,1164,1342,1143,778,804,619,452,365,1196,471,1263,270,1292,631,552,327,1255,855,660,603,353,670,960,500,643,666,339,520,515,1001,829,275,804,343,485,343,485,370,1008,624,337,971,855,637,871,1095,672,695,5,1578,637,972,1342,1003,702],
                    [804,981,1044,597,144,340,457,538,1029,379,945,170,863,368,898,1044,1027,423,871,291,385,717,831,136,950,299,1121,356,619,370,1070,595,981,329,343,395,743,411,643,423,170,277,650,1029,506,329,728,410,546,410,546,286,776,349,404,747,595,958,597,978,429,367,415,1474,958,801,1044,966,769],
                    [1126,1843,407,1146,1201,1140,1103,1404,671,1414,655,1117,753,1168,576,407,515,913,881,1142,1064,1579,776,1138,407,1162,334,1394,1067,1170,390,1310,1843,1367,1100,1433,703,956,1556,879,1117,1000,1269,671,925,1162,656,1068,1013,1068,1013,1094,1186,941,1068,1294,1310,1820,1544,466,885,1010,1277,2073,1820,553,407,470,813],
                    [901,794,1359,1013,728,512,699,592,1327,568,1146,553,1161,427,1056,1359,1325,900,1286,844,694,662,989,696,1254,695,1419,468,1041,483,1385,545,794,393,730,173,1087,708,361,831,553,729,1029,1327,962,727,1026,826,894,826,894,660,649,747,820,593,545,771,446,1276,837,920,711,1154,771,1099,1359,1264,1184],
                    [1269,271,1258,1016,966,751,937,671,1487,878,1514,791,1296,665,1424,1258,1563,1139,1290,1082,937,446,1357,934,1146,756,1657,706,1038,722,1143,913,271,678,839,554,1326,947,220,1069,791,967,925,1487,1216,761,1290,828,971,828,971,856,1017,985,822,961,913,277,814,1514,1075,1158,650,1273,277,1337,1258,1489,1188],
                    [580,1307,598,655,542,582,396,846,735,707,539,491,569,610,449,598,730,123,729,283,506,1042,382,479,504,604,824,768,630,612,624,833,1307,741,542,807,297,397,998,140,491,261,849,735,218,604,394,593,455,593,455,536,710,256,593,811,833,1284,967,681,146,192,741,1864,1284,504,598,669,627],
                    [486,1168,824,806,324,557,0,727,1045,358,731,363,879,471,641,824,1011,501,1039,371,487,1022,575,382,730,540,1072,446,834,528,850,493,1168,419,523,485,523,501,742,538,363,395,830,1045,372,560,709,619,703,619,703,533,458,467,613,464,493,1145,630,962,544,346,656,1499,1145,779,824,950,937],
                    [701,934,1006,550,184,292,276,491,995,226,913,74,829,289,823,1006,993,538,824,399,341,670,756,251,912,252,1087,203,572,322,1032,442,934,176,295,242,705,389,490,446,74,300,602,995,474,281,694,363,498,363,498,238,595,372,357,558,442,911,444,944,452,475,368,1321,911,767,1006,932,722],
                    [1288,565,1467,632,659,499,863,406,1125,764,1368,464,934,486,1289,1467,1267,902,916,744,576,195,1320,596,1387,337,1416,721,630,441,1379,980,565,693,478,693,1084,624,633,791,464,645,506,1125,954,400,928,467,609,467,609,495,1170,748,461,1114,980,542,961,1219,797,820,288,1568,542,1096,1467,1127,814],
                    [691,1001,997,617,148,360,266,558,1062,267,904,142,896,328,814,997,1060,498,891,366,409,737,747,259,903,319,1154,297,639,390,1023,483,1001,270,363,336,696,457,584,514,142,368,670,1062,447,349,761,430,566,430,566,306,591,440,424,555,483,978,538,1011,520,442,435,1415,978,834,997,999,789],
                    [1289,1237,898,416,891,778,1094,917,448,996,1111,696,319,840,1021,898,590,673,88,902,665,973,1060,828,813,604,763,972,337,778,805,1212,1237,945,595,1011,726,628,1219,639,696,760,457,448,927,587,436,461,489,461,489,682,1365,700,461,1328,1212,1214,1213,543,645,885,671,2060,1214,616,898,438,30],
                    [848,930,1306,1038,756,624,635,704,1439,515,1093,563,1273,539,1003,1306,1437,1012,1312,888,806,774,936,740,1201,740,1531,580,1060,595,1332,492,930,341,784,293,1117,820,434,943,563,789,1091,1439,909,770,1138,851,1006,851,1006,727,596,859,845,540,492,883,394,1388,949,906,823,1049,883,1211,1306,1376,1210],
                    [1112,906,1106,289,560,447,764,585,760,665,1059,365,568,508,980,1106,902,576,562,624,335,641,994,497,1021,273,1050,641,311,446,1013,881,906,614,265,680,718,307,888,499,365,454,463,760,782,255,563,58,244,58,244,350,1034,449,24,997,881,883,882,854,505,656,340,1729,883,730,1106,761,460],
                    [502,1208,960,1012,488,724,363,894,1266,413,747,535,1100,638,657,960,1187,709,1285,577,693,1077,591,587,855,712,1208,580,1040,695,986,426,1208,479,729,596,744,707,786,747,535,601,1036,1266,578,742,930,825,909,825,909,699,354,673,819,396,426,1185,551,1138,753,552,828,1420,1185,904,960,1130,1183],
                    [1284,1461,772,741,1012,951,1156,1141,0,1228,985,928,333,979,895,772,142,724,463,953,841,1197,934,949,687,828,613,1205,597,981,679,1444,1461,1178,820,1244,650,686,1367,690,928,811,728,0,978,811,401,686,664,686,664,905,1344,751,686,1452,1444,1438,1446,202,696,936,895,2233,1438,501,772,223,395],
                    [599,1063,1057,917,501,659,380,829,1361,254,844,441,1195,573,754,1057,1284,816,1191,684,708,931,687,612,952,619,1305,487,939,630,1083,231,1063,334,662,450,869,756,640,813,441,667,969,1361,660,648,1060,730,865,730,865,605,347,739,724,292,231,1040,406,1235,819,659,735,1274,1040,1001,1057,1227,1089],
                    [592,1220,696,641,461,524,420,788,803,709,597,409,637,552,518,696,799,137,797,194,448,956,515,398,602,538,893,686,663,554,722,845,1220,659,484,725,395,329,940,148,409,179,791,803,242,521,463,556,438,556,438,478,722,165,556,835,845,1197,927,750,154,95,654,1806,1197,573,696,738,695],
                    [1124,860,1340,404,532,419,686,593,876,637,1241,337,737,447,1162,1340,1018,795,687,617,469,542,1152,469,1247,183,1242,613,401,449,1231,853,860,586,413,652,955,517,835,664,337,518,271,876,827,345,753,380,522,380,522,356,1006,641,374,969,853,837,854,968,670,693,343,1701,837,933,1340,917,585],
                    [1181,410,1569,756,761,616,857,415,1227,798,1426,566,1036,593,1336,1569,1369,1004,1030,846,678,278,1269,698,1489,490,1518,626,778,554,1481,825,410,598,580,466,1186,726,406,893,566,747,659,1227,1056,502,1030,569,711,569,711,597,929,850,563,873,825,387,727,1321,899,922,390,1341,387,1198,1569,1229,928],
                    [762,934,1003,550,163,298,415,491,922,423,904,123,756,326,825,1003,920,419,824,249,284,670,789,94,909,252,1014,400,572,328,1029,639,934,373,296,439,684,298,714,327,123,181,603,922,464,282,621,363,489,363,489,245,734,253,357,705,639,911,641,871,333,325,368,1518,911,694,1003,859,722],
                    [963,693,1361,945,660,445,631,490,1260,572,1208,485,1094,359,1118,1361,1257,833,1218,776,626,561,1051,628,1316,628,1351,400,973,416,1314,607,693,372,662,248,1020,641,209,763,485,661,928,1260,910,659,958,758,826,758,826,592,711,679,752,655,607,670,508,1208,769,852,641,1144,670,1031,1361,1196,1116],
                    [2317,1343,2443,2009,2014,1799,1985,1743,2533,1926,2562,1839,2343,1713,2472,2443,2611,2187,2293,2130,1980,1382,2405,1982,2331,1715,2705,1754,2007,1770,2328,1961,1343,1726,1911,1602,2374,1995,1268,2117,1839,2015,1890,2533,2264,1833,2359,1900,2042,1900,2042,1912,2065,2033,1894,2009,1961,1349,1862,2562,2123,2206,1721,2006,1349,2385,2443,2523,2191],
                    [906,744,1122,360,314,201,468,348,831,419,1023,119,640,229,944,1122,933,538,634,399,241,480,934,251,1028,62,1027,395,382,231,990,635,744,368,195,434,696,289,617,446,119,300,392,831,609,105,634,173,315,173,315,138,788,355,167,751,635,721,636,884,452,475,178,1483,721,707,1122,833,532],
                    [1462,0,1520,1043,1047,902,1138,699,1514,1079,1707,852,1323,866,1617,1520,1656,1291,1317,1132,965,474,1550,984,1408,783,1805,907,1065,840,1405,1106,0,879,867,747,1473,1013,515,1179,852,1033,953,1514,1342,789,1317,856,998,856,998,884,1210,1137,850,1154,1106,354,1008,1608,1185,1208,677,1568,354,1485,1520,1516,1215],
                    [868,1520,0,1134,1007,1151,845,1415,818,1156,397,1060,920,1207,318,0,662,692,1028,852,1075,1567,518,1049,284,1173,498,1198,1055,1181,127,1052,1831,1214,1111,1237,505,966,1478,709,1060,830,1337,818,667,1173,750,1056,1024,1056,1024,1105,928,816,1056,1036,1052,1808,1286,613,715,752,1265,2155,1808,547,0,617,960],
                    [1184,1043,1134,0,679,566,882,704,725,784,1093,484,535,627,1014,1122,867,649,485,725,453,760,1067,616,1045,392,1040,760,196,565,1029,1000,1025,733,383,799,753,426,1007,572,484,573,478,725,834,374,551,249,296,249,296,469,1153,501,249,1116,1000,1002,1001,820,578,708,458,1848,1002,731,1122,715,383],
                    [725,1047,1007,679,0,415,300,629,1039,416,915,261,873,475,825,1013,1037,392,979,260,387,808,758,153,919,390,1131,392,734,445,1039,631,1072,365,423,431,712,401,746,441,261,295,730,1039,433,430,738,519,606,519,606,362,627,367,513,590,631,1049,756,988,447,336,506,1612,1049,811,1013,976,877],
                    [855,902,1151,566,415,0,507,459,860,415,941,115,694,214,862,1040,857,433,819,376,227,646,851,228,922,228,951,391,574,216,914,631,910,364,263,430,620,241,602,364,115,262,570,860,558,259,559,359,427,359,427,192,784,280,353,747,631,887,632,808,370,452,344,1469,887,631,1040,796,717],
                    [486,1138,845,882,300,507,0,727,1045,358,731,363,879,471,641,824,1011,501,1039,371,487,1022,575,382,730,540,1072,446,834,528,850,493,1168,419,523,485,523,501,742,538,363,395,830,1045,372,560,709,619,703,619,703,533,458,467,613,464,493,1145,630,962,544,346,656,1499,1145,779,824,950,937],
                    [1154,699,1415,704,629,459,727,0,1216,732,1366,445,1025,327,1276,1336,1154,729,1019,725,515,551,1209,577,1219,485,1248,587,767,283,1211,932,774,559,469,626,916,563,571,660,445,654,692,1216,950,455,1019,557,672,557,672,412,1036,576,551,980,932,751,827,1105,666,801,406,1506,751,928,1336,1218,917],
                    [1284,1514,818,725,1039,860,1045,1216,0,1228,985,928,333,979,895,772,142,724,463,953,841,1197,934,949,687,828,613,1205,597,981,679,1444,1461,1178,820,1244,650,686,1367,690,928,811,728,0,978,811,401,686,664,686,664,905,1344,751,686,1452,1444,1438,1446,202,696,936,895,2233,1438,501,772,223,395],
                    [755,1079,1156,784,416,415,358,732,1228,0,1000,308,1062,461,910,1124,1226,769,1057,637,575,903,843,485,1030,485,1320,375,805,518,1150,276,1088,348,529,414,823,623,662,680,308,534,836,1228,672,515,927,596,732,596,732,472,503,606,590,447,276,1065,542,1177,686,646,601,1410,1065,1000,1124,1165,955],
                    [623,1707,397,1093,915,941,731,1366,985,1000,0,958,844,1064,74,527,753,612,1067,875,991,1488,273,886,422,1087,737,1070,976,1101,553,807,1698,969,1027,1086,425,886,1276,629,958,751,1258,909,530,1064,674,977,944,977,944,1025,683,737,977,791,807,1675,1041,704,635,673,1186,1872,1675,471,527,696,998],
                    [774,852,1060,484,261,115,363,445,928,308,958,0,754,246,869,1047,918,463,749,324,267,595,829,176,953,177,1012,276,497,248,975,515,859,249,221,315,681,315,563,371,0,225,528,920,534,207,619,288,424,288,424,164,669,297,282,632,515,836,517,869,377,400,293,1394,836,692,1047,857,647],
                    [1150,1323,920,535,873,694,879,1025,333,1062,844,754,0,789,803,778,443,534,304,763,654,961,842,759,693,592,643,960,429,766,685,1200,1225,933,584,999,550,496,1177,500,684,621,691,302,788,575,208,450,441,450,441,670,1252,561,450,1360,1200,1202,1201,396,506,746,659,2043,1202,388,778,318,235],
                    [936,866,1207,627,475,214,471,327,979,461,1064,246,789,0,943,1121,939,514,900,458,308,628,932,310,1004,309,1033,341,655,98,996,712,892,314,344,380,701,323,496,445,196,343,651,941,639,341,640,440,508,440,508,274,790,361,434,734,712,869,582,890,451,534,326,1362,869,713,1121,878,798],
                    [548,1617,318,1014,825,862,641,1276,895,910,74,869,803,943,0,452,678,538,992,800,916,1414,198,811,347,1012,700,995,902,1027,478,732,1623,894,952,1011,351,812,1201,555,883,677,1184,834,455,989,599,903,870,903,870,951,608,663,903,716,732,1600,966,629,561,599,1112,1835,1600,396,452,621,923],
                    [868,1520,0,1122,1013,1040,824,1336,772,1124,527,1047,778,1121,452,0,662,692,1028,852,1075,1567,518,1049,284,1173,498,1198,1055,1181,127,1052,1831,1214,1111,1237,505,966,1478,709,1060,830,1337,818,667,1173,750,1056,1024,1056,1024,1105,928,816,1056,1036,1052,1808,1286,613,715,752,1265,2155,1808,547,0,617,960],
                    [1097,1656,662,867,1037,857,1011,1154,142,1226,753,918,443,939,678,662,0,674,616,903,825,1350,747,899,500,923,408,1155,750,931,492,1281,1614,1128,861,1194,463,716,1317,640,878,761,881,153,794,923,416,839,773,839,773,855,1157,701,839,1265,1281,1591,1396,75,646,886,1048,2183,1591,313,585,177,548],
                    [691,1291,692,649,392,433,501,729,724,769,612,463,534,514,538,692,674,0,725,213,472,980,492,373,588,562,820,710,626,578,708,944,1244,683,508,749,381,353,964,136,433,203,815,731,329,545,390,580,451,580,451,502,821,189,580,922,944,1221,951,677,126,196,678,1830,1221,500,682,665,623],
                    [1382,1317,1028,485,979,819,1039,1019,463,1057,1067,749,304,900,992,1028,616,725,0,995,749,1057,1089,912,842,688,792,1056,421,862,834,1296,1321,1029,679,1095,819,712,1303,732,780,853,527,477,1020,671,465,545,573,545,573,766,1499,793,545,1412,1296,1298,1297,572,738,978,755,2144,1298,645,927,467,118],
                    [737,1132,852,725,260,376,371,725,953,637,875,324,763,458,800,852,903,213,995,0,377,890,660,246,747,472,983,620,724,499,867,860,1154,593,413,659,540,304,885,238,343,159,720,893,387,450,553,509,527,509,527,423,744,254,503,819,860,1131,861,840,244,164,588,1752,1131,663,841,828,785],
                    [856,965,1075,453,387,227,487,515,841,575,991,267,654,308,916,1075,825,472,749,377,0,645,862,305,933,277,962,552,456,328,925,791,910,525,145,591,631,128,714,375,275,263,452,790,559,182,600,239,317,239,317,190,828,291,242,910,791,887,793,819,381,444,344,1581,887,643,1051,784,599],
                    [1312,474,1567,760,808,646,1022,551,1197,903,1488,595,961,628,1414,1567,1350,980,1057,890,645,0,1400,709,1500,450,1529,756,743,554,1492,956,449,728,591,597,1197,737,537,904,577,758,619,1238,1067,513,1041,580,722,580,722,608,1060,861,574,1004,956,426,857,1332,910,933,401,1472,426,1209,1580,1240,927],
                    [488,1550,518,1067,758,851,575,1209,934,843,273,829,842,932,198,518,747,492,1089,660,862,1400,0,752,349,953,702,936,900,984,480,672,1564,835,893,952,350,769,1142,512,824,633,1200,837,395,930,601,901,827,901,827,908,549,628,901,656,672,1541,907,632,518,564,1069,1776,1541,398,454,624,925],
                    [734,984,1049,616,153,228,382,577,949,485,886,176,759,310,811,1049,899,373,912,246,305,709,752,0,880,315,1055,463,625,369,1000,719,997,436,314,502,673,292,755,321,186,175,621,963,436,354,662,410,530,410,530,287,706,247,404,677,719,974,704,912,327,297,431,1581,974,735,974,900,768],
                    [792,1408,284,1045,919,922,730,1219,687,1030,422,953,693,1004,347,284,500,588,842,747,933,1500,349,880,0,1037,443,1129,941,1045,142,976,1718,1139,975,1256,435,830,1247,639,992,761,1223,675,598,1037,523,943,887,943,887,969,852,747,943,960,976,1695,1211,470,645,683,1152,1854,1695,321,257,462,809],
                    [965,783,1173,392,390,228,540,485,828,485,1087,177,592,309,1012,1173,923,562,688,472,277,450,953,315,1037,0,1086,454,352,290,1049,694,750,427,254,493,755,348,676,505,178,359,370,870,668,145,673,212,354,212,354,197,847,414,206,810,694,727,695,943,511,534,184,1542,727,766,1181,872,536],
                    [1173,1805,498,1040,1131,951,1072,1248,613,1320,737,1012,643,1033,700,498,408,820,792,983,962,1529,702,1055,443,1086,0,1279,952,1055,495,1357,1728,1252,985,1318,588,841,1441,764,1002,885,1154,556,919,1047,541,953,898,953,898,979,1233,826,953,1341,1357,1705,1520,351,770,1010,1162,2069,1705,438,512,355,698],
                    [928,907,1198,760,392,391,446,587,1205,375,1070,276,960,341,995,1198,1155,710,1056,620,552,756,936,463,1129,454,1279,0,811,399,1237,617,944,338,535,270,910,624,518,686,314,540,842,1243,759,521,941,602,738,602,738,478,752,662,596,715,617,921,606,1191,692,715,607,1384,921,1014,1211,1179,961],
                    [1186,1065,1055,196,734,574,834,767,597,805,976,497,429,655,902,1055,750,626,421,724,456,743,900,625,941,352,952,811,0,567,946,1002,1027,735,385,801,670,427,1009,573,486,575,319,598,836,376,468,251,298,251,298,471,1155,503,251,1118,1002,1004,1003,690,579,710,460,1850,1004,648,1039,631,299],
                    [941,840,1181,565,445,216,528,283,981,518,1101,248,766,98,1027,1181,931,578,862,499,328,554,984,369,1045,290,1055,399,567,0,1024,719,831,346,298,413,729,350,502,473,224,371,600,969,667,284,668,381,501,381,501,241,823,389,375,767,719,808,614,918,479,561,265,1368,808,741,1149,906,741],
                    [885,1405,127,1029,1039,914,850,1211,679,1150,553,975,685,996,478,127,492,708,834,867,925,1492,480,1000,142,1049,495,1237,946,1024,0,1069,1848,1231,1128,1254,522,983,1495,726,1077,847,1354,840,684,1190,767,1073,1041,1073,1041,1122,945,833,1073,1053,1069,1825,1303,635,732,769,1282,2172,1825,564,166,639,982],
                    [629,1106,1052,1000,631,631,493,932,1444,276,807,515,1200,712,732,1052,1281,944,1296,860,791,956,672,719,976,694,1357,617,1002,719,1069,0,1166,437,757,553,898,851,743,908,536,762,1064,1456,690,743,1234,824,960,824,960,700,377,834,818,201,0,1143,509,1265,914,797,829,1377,1143,1031,1087,1257,1183],
                    [1462,0,1831,1025,1072,910,1168,774,1461,1088,1698,859,1225,892,1623,1831,1614,1244,1321,1154,910,449,1564,997,1718,750,1728,944,1027,831,1848,1166,0,879,867,747,1473,1013,515,1179,852,1033,953,1514,1342,789,1317,856,998,856,998,884,1210,1137,850,1154,1106,354,1008,1608,1185,1208,677,1568,354,1485,1520,1516,1215],
                    [765,879,1214,733,365,364,419,559,1178,348,969,249,933,314,894,1214,1128,683,1029,593,525,728,835,436,1139,427,1252,338,735,346,1231,437,879,0,502,237,877,591,485,653,281,507,809,1210,726,488,908,569,705,569,705,445,513,629,563,457,409,888,304,1158,659,682,574,1182,888,981,1223,1146,928],
                    [998,867,1111,383,423,263,523,469,820,529,1027,221,584,344,952,1111,861,508,679,413,145,591,893,314,975,254,985,535,385,298,1128,757,867,502,0,539,712,306,747,493,224,405,307,754,701,34,557,96,238,96,238,165,893,443,90,856,740,742,741,848,499,580,199,1588,742,724,1100,755,454],
                    [872,747,1237,799,431,430,485,626,1244,414,1086,315,999,380,1011,1237,1194,749,1095,659,591,597,952,502,1256,493,1318,270,801,413,1254,553,747,237,539,0,902,616,341,678,306,532,834,1235,751,513,933,594,730,594,730,470,620,654,588,564,516,740,418,1183,684,707,599,1124,740,1006,1203,1171,953],
                    [636,1473,505,753,712,620,523,916,650,823,425,681,550,701,351,505,463,381,819,540,631,1197,350,673,435,755,588,910,670,729,522,898,1473,877,712,902,0,539,1140,373,701,494,905,672,331,746,284,624,596,624,596,678,749,480,624,856,872,1376,1080,467,379,416,833,2006,1376,290,485,455,619],
                    [870,1013,966,426,401,241,501,563,686,623,886,315,496,323,812,966,716,353,712,304,128,737,769,292,830,348,841,624,427,350,983,851,1013,591,306,616,539,0,738,270,299,167,539,662,520,269,472,298,233,298,233,277,883,153,298,925,815,974,817,715,276,393,431,1604,974,538,946,656,559],
                    [1049,515,1478,1007,746,602,742,571,1367,662,1276,563,1177,496,1201,1478,1317,964,1303,885,714,537,1142,755,1247,676,1441,518,1009,502,1495,743,515,485,747,341,1140,738,0,849,571,747,932,1346,996,745,1044,824,912,824,912,678,797,765,818,741,693,502,594,1294,855,938,645,1053,502,1117,1447,1282,1183],
                    [720,1179,709,572,441,364,538,660,690,680,629,371,500,445,555,709,640,136,732,238,375,904,512,321,639,505,764,686,573,473,726,908,1179,653,493,678,373,270,849,0,363,133,729,666,358,484,326,465,327,465,327,416,850,128,465,956,879,1151,881,613,6,259,608,1744,1151,436,711,601,558],
                    [774,852,1060,484,261,115,363,445,928,308,958,0,684,196,883,1060,878,433,780,343,275,577,824,186,992,178,1002,314,486,224,1077,536,852,281,224,306,701,299,571,363,0,225,528,920,534,207,619,288,424,288,424,164,669,297,282,632,515,836,517,869,377,400,293,1394,836,692,1047,857,647],
                    [717,1033,830,573,295,262,395,654,811,534,751,225,621,343,677,830,761,203,853,159,263,758,633,175,761,359,885,540,575,371,847,762,1033,507,405,532,494,167,747,133,225,0,611,801,367,341,460,400,383,400,383,314,755,71,394,823,745,1017,747,747,151,241,474,1643,1017,570,821,735,693],
                    [1261,953,1337,478,730,570,830,692,728,836,1258,528,691,651,1184,1337,881,815,527,720,452,619,1200,621,1223,370,1154,842,319,600,1354,1064,953,809,307,834,905,539,932,729,528,611,0,694,964,402,709,440,539,440,539,516,1174,702,434,1137,1021,976,1022,786,792,849,514,1869,976,889,1239,779,448],
                    [1284,1514,818,725,1039,860,1045,1216,0,1228,909,920,302,941,834,818,153,731,477,893,790,1238,837,963,675,870,556,1243,598,969,840,1456,1514,1210,754,1235,672,662,1346,666,920,801,694,0,978,811,401,686,664,686,664,905,1344,751,686,1452,1444,1438,1446,202,696,936,895,2233,1438,501,772,223,395],
                    [535,1342,667,834,433,558,372,950,978,672,530,534,788,639,455,667,794,329,1020,387,559,1067,395,436,598,668,919,759,836,667,684,690,1342,726,701,751,331,520,996,358,534,367,964,978,0,557,832,616,700,616,700,530,507,464,610,592,622,1213,748,963,544,343,695,1617,1213,780,825,951,974],
                    [947,789,1173,374,430,259,560,455,811,515,1064,207,575,341,989,1173,923,545,671,450,182,513,930,354,1037,145,1047,521,376,284,1190,743,789,488,34,513,746,269,745,484,207,341,402,811,557,0,648,187,305,187,305,131,895,388,181,858,742,794,743,917,478,535,251,1590,794,740,1149,847,546],
                    [925,1317,750,551,738,559,709,1019,401,927,674,619,208,640,599,750,416,390,465,553,600,1041,601,662,523,673,541,941,468,668,767,1234,1317,908,557,933,284,472,1044,326,619,460,709,401,832,648,0,505,496,505,496,610,1105,457,505,1213,1229,1257,1151,433,401,641,714,1938,1257,240,632,278,388],
                    [1052,856,1056,249,519,359,619,557,686,596,977,288,450,440,903,1056,839,580,545,509,239,580,901,410,943,212,953,602,251,381,1073,824,856,569,96,594,624,298,824,465,288,400,440,686,616,187,505,0,349,0,349,290,974,493,69,937,821,823,822,940,583,640,280,1669,823,851,1254,835,503],
                    [1072,998,1024,296,606,427,703,672,664,732,944,424,441,508,870,1024,773,451,573,527,317,722,827,530,887,354,898,738,298,501,1041,960,998,705,238,730,596,233,912,327,424,383,539,664,700,305,496,349,0,139,0,359,1042,389,139,1005,889,891,890,762,443,596,348,1737,891,638,1014,669,368],
                    [1052,856,1056,249,519,359,619,557,686,596,977,288,450,440,903,1056,839,580,545,509,239,580,901,410,943,212,953,602,251,381,1073,824,856,569,96,594,624,298,824,465,288,400,440,686,616,187,505,0,139,0,349,290,974,493,69,937,821,823,822,940,583,640,280,1669,823,851,1254,835,503],
                    [1072,998,1024,296,606,427,703,672,664,732,944,424,441,508,870,1024,773,451,573,527,317,722,827,530,887,354,898,738,298,501,1041,960,998,705,238,730,596,233,912,327,424,383,539,664,700,305,496,349,0,349,0,359,1042,389,139,1005,889,891,890,762,443,596,348,1737,891,638,1014,669,368],
                    [833,884,1105,469,362,192,533,412,905,472,1025,164,670,274,951,1105,855,502,766,423,190,608,908,287,969,197,979,478,471,241,1122,700,884,445,165,470,678,277,678,416,164,314,516,905,530,131,610,290,359,290,359,0,805,274,312,887,768,956,770,803,364,421,413,1558,956,626,1035,791,684],
                    [481,1210,928,1153,627,784,458,1036,1344,503,683,669,1252,790,608,928,1157,821,1499,744,828,1060,549,706,852,847,1233,752,1155,823,945,377,1210,513,893,620,749,883,797,850,669,755,1174,1344,507,895,1105,974,1042,974,1042,805,0,771,916,342,358,1242,608,1117,851,649,936,1476,1242,883,939,1109,1280],
                    [709,1137,816,501,367,280,467,576,751,606,737,297,561,361,663,816,701,189,793,254,291,861,628,247,747,414,826,662,503,389,833,834,1137,629,443,654,480,153,765,128,297,71,702,751,464,388,457,493,389,493,389,274,771,0,420,933,855,1064,857,690,137,232,521,1644,1064,513,813,678,632],
                    [1112,850,1056,249,513,353,613,551,686,590,977,282,450,434,903,1056,839,580,545,503,242,574,901,404,943,206,953,596,251,375,1073,818,850,563,90,588,624,298,818,465,282,394,434,686,610,181,505,69,139,69,139,312,916,420,0,997,881,883,882,854,505,656,340,1729,883,730,1106,761,460],
                    [555,1154,1036,1116,590,747,464,980,1452,447,791,632,1360,734,716,1036,1265,922,1412,819,910,1004,656,677,960,810,1341,715,1118,767,1053,201,1154,457,856,564,856,925,741,956,632,823,1137,1452,592,858,1213,937,1005,937,1005,887,342,933,997,0,201,1165,531,1191,926,724,927,1399,1165,957,1013,1183,1281],
                    [629,1106,1052,1000,631,631,493,932,1444,276,807,515,1200,712,732,1052,1281,944,1296,860,791,956,672,719,976,694,1357,617,1002,719,1069,0,1106,409,740,516,872,815,693,879,515,745,1021,1444,622,742,1229,821,889,821,889,768,358,855,881,201,0,1143,509,1265,914,797,829,1377,1143,1031,1087,1257,1183],
                    [1421,354,1808,1002,1049,887,1145,751,1438,1065,1675,836,1202,869,1600,1808,1591,1221,1298,1131,887,426,1541,974,1695,727,1705,921,1004,808,1825,1143,354,888,742,740,1376,974,502,1151,836,1017,976,1438,1213,794,1257,823,891,823,891,956,1242,1064,883,1165,1143,0,967,1567,1144,1167,636,1581,0,1444,1814,1475,1174],
                    [823,1008,1286,1001,756,632,630,827,1446,542,1041,517,1201,582,966,1286,1396,951,1297,861,793,857,907,704,1211,695,1520,606,1003,614,1303,509,1008,304,741,418,1080,817,594,881,517,747,1022,1446,748,743,1151,822,890,822,890,770,608,857,882,531,509,967,0,1406,906,929,821,966,1016,1229,1281,1394,1175],
                    [1082,1608,613,820,988,808,962,1105,202,1177,704,869,396,890,629,613,75,677,572,840,819,1332,632,912,470,943,351,1191,690,918,635,1265,1608,1158,848,1183,467,715,1294,613,869,747,786,202,963,917,433,940,762,940,762,803,1117,690,854,1191,1265,1567,1406,0,631,871,999,2168,1542,299,570,102,499],
                    [728,1185,715,578,447,370,544,666,696,686,635,377,506,451,561,715,646,126,738,244,381,910,518,327,645,511,770,692,579,479,732,914,1185,659,499,684,379,276,855,6,377,151,792,696,544,478,401,583,443,583,443,364,851,137,505,926,914,1144,906,631,0,267,616,1752,1159,444,719,609,566],
                    [709,1208,752,708,336,452,346,801,936,646,673,400,746,534,599,752,886,196,978,164,444,933,564,297,683,534,1010,715,710,561,769,797,1208,682,580,707,416,393,938,259,400,241,849,936,343,535,641,640,596,640,596,421,649,232,656,724,797,1167,929,871,267,0,623,1787,1166,635,813,800,757],
                    [1130,677,1265,458,506,344,656,406,895,601,1186,293,659,326,1112,1265,1048,678,755,588,344,401,1069,431,1152,184,1162,607,460,265,1282,829,677,574,199,599,833,431,645,608,293,474,514,895,695,251,714,280,348,280,348,413,936,521,340,927,829,636,821,999,616,623,0,1583,642,976,1346,1007,706],
                    [1667,1568,2155,1848,1612,1469,1499,1506,2233,1410,1872,1394,2043,1362,1835,2155,2183,1830,2144,1752,1581,1472,1776,1581,1854,1542,2069,1384,1850,1368,2172,1377,1568,1182,1588,1124,2006,1604,1053,1744,1394,1643,1869,2233,1617,1590,1938,1669,1737,1669,1737,1558,1476,1644,1729,1399,1377,1581,966,2168,1752,1787,1583,0,1533,1941,2125,2106,2026],
                    [1421,354,1808,1002,1049,887,1145,751,1438,1065,1675,836,1202,869,1600,1808,1591,1221,1298,1131,887,426,1541,974,1695,727,1705,921,1004,808,1825,1143,354,888,742,740,1376,974,502,1151,836,1017,976,1438,1213,794,1257,823,891,823,891,956,1242,1064,883,1165,1143,0,1016,1542,1159,1166,642,1533,0,1444,1814,1475,1174],
                    [800,1485,547,731,811,631,779,928,501,1000,471,692,388,713,396,547,313,500,645,663,643,1209,398,735,321,766,438,1014,648,741,564,1031,1485,981,724,1006,290,538,1117,436,692,570,889,501,780,740,240,851,638,851,638,626,883,513,730,957,1031,1444,1229,299,444,635,976,1941,1444,0,461,250,522],
                    [868,1520,0,1122,1013,1040,824,1336,772,1124,527,1047,778,1121,452,0,585,682,927,841,1051,1580,454,974,257,1181,512,1211,1039,1149,166,1087,1520,1223,1100,1203,485,946,1447,711,1047,821,1239,772,825,1149,632,1254,1014,1254,1014,1035,939,813,1106,1013,1087,1814,1281,570,719,813,1346,2125,1814,461,0,617,960],
                    [1016,1516,617,715,976,796,950,1218,223,1165,696,857,318,878,621,617,177,665,467,828,784,1240,624,900,462,872,355,1179,631,906,639,1257,1516,1146,755,1171,455,656,1282,601,857,735,779,223,951,847,278,835,669,835,669,791,1109,678,761,1183,1257,1475,1394,102,609,800,1007,2106,1475,250,617,0,388],
                    [1277,1215,960,383,877,717,937,917,395,955,998,647,235,798,923,960,548,623,118,785,599,927,925,768,809,536,698,961,299,741,982,1183,1215,928,454,953,619,559,1183,558,647,693,448,395,974,546,388,503,368,503,368,684,1280,632,460,1281,1183,1174,1175,499,566,757,706,2026,1174,522,960,388,0]]
   return data

In [ ]:
import re

root = ['x3(3,264)', 'x3(264,0)', 'x5(5,205)', 'x5(190,242)', 'x5(205,190)', 'x5(242,282)', 'x5(282,0)', 'x13(13,248)', 'x13(160,229)', 'x13(229,0)', 'x13(248,160)', 'x14(14,0)', 'x17(17,208)', 'x17(196,0)', 'x17(208,196)', 'x26(26,157)', 'x26(157,301)', 'x26(168,222)', 'x26(222,0)', 'x26(301,168)', 'x29(29,245)', 'x29(245,0)', 'x40(40,237)', 'x40(235,0)', 'x40(237,274)', 'x40(249,235)', 'x40(274,249)', 'x41(41,0)', 'x44(44,177)', 'x44(177,0)']


data = create_data_model()
X = []
for r in root:
    m = re.findall(r'\d+', r)
    is_exist = False
    for x in X:
        if x[0] == m[0]:
            is_exist = True
            x.append(m[1])
    if not is_exist:
        X.append([m[0]])

for i in range(len(X)):
    X[i][0] = data['Vs'].index(int(X[i][0]))
for i in range(len(X)):
    for j in range(1, len(X[i])):
        X[i][j] = data['Vc'].index(int(X[i][j]))
print(X)

jaccard = [[1,0.1666666667,0.4,0.1666666667,0.2,0.4,0.2,0.4,0.2,0,0.6,0.2,0.3333333333,0.4,0.6,0.75,0.4,0.25,0.2,0.2,0],[1,0.75,0.1666666667,0.75,0.2,0.75,0.2,0.75,0.2,0.5,0.3333333333,0.5,1,0.4,0.3333333333,0.1666666667,0.75,0,0.5,0.2,0.5],[1,0.25,0.25,0,0.3333333333,0.25,0.3333333333,0.25,0.3333333333,0,0.5,0,0.2,0.6666666667,0.2,0.6666666667,0.25,0.5,0,0.3333333333,0],[1,0.6666666667,0,0.25,0.3333333333,0.6666666667,0.3333333333,0.6666666667,0.3333333333,0.3333333333,0.2,0.3333333333,0.5,0.25,0,0.25,0.6666666667,0,0.3333333333,0,0.3333333333],[1,0.2,0,0.2,0.25,0.5,0.6666666667,0.5,0.6666666667,0,0.1666666667,0.25,0.4,0.2,0.1666666667,0.2,0.5,0,0.25,0,0],[1,1,0.2,0.5,0.25,0.5,0.25,0.5,0.25,0.6666666667,0.4,0.25,0.75,0.5,0.1666666667,0.2,0.5,0,0.25,0.25,0.6666666667],[1,0,0.3333333333,0,0,0,0,0,0,0,0.25,0,0,0.3333333333,0.25,0.3333333333,0,1,0,0.5,0],[1,0.6666666667,0,0.25,0.3333333333,0.6666666667,0.3333333333,0.6666666667,0.3333333333,0.3333333333,0.2,0.3333333333,0.5,0.25,0,0.25,0.6666666667,0,0.3333333333,0,0.3333333333],[1,0.75,0.1666666667,0.75,0.2,0.75,0.2,0.75,0.2,0.5,0.3333333333,0.5,1,0.4,0.3333333333,0.1666666667,0.75,0,0.5,0.2,0.5],[1,0,0,0,0.5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[1,0,0.2,0.5,0.25,0.5,0.25,0.5,0.25,0.6666666667,0.4,0.25,0.75,0.5,0.1666666667,0.2,0.5,0,0.25,0.25,0.6666666667],[1,0.2,0,0.2,0,0,0,0,0,0.25,0.75,0,0.1666666667,0.5,0.75,0.5,0,0.3333333333,0,0.6666666667,0.25],[1,0.5,0.2,0,0,0.5,0,0.5,0,0.6666666667,0.1666666667,0.6666666667,0.75,0.2,0.4,0,0.5,0,0.6666666667,0.25,0.6666666667],[1,0.25,0,0,0,0.25,0.3333333333,0.25,0.3333333333,0,0.2,0,0.2,0.25,0,0.25,0.25,0,0,0,0],[1,0.5,0,0.5,0.25,0,0.25,1,0.25,0.25,0.1666666667,0.6666666667,0.75,0.2,0.1666666667,0.2,1,0,0.6666666667,0,0.25],[1,0.25,0,0,0.3333333333,0.25,0,0.25,1,0,0.2,0,0.2,0.25,0,0.25,0.25,0,0,0,0],[1,0.5,0,0.5,0.25,1,0.25,0,0.25,0.25,0.1666666667,0.6666666667,0.75,0.2,0.1666666667,0.2,1,0,0.6666666667,0,0.25],[1,0.25,0,0,0.3333333333,0.25,1,0.25,0,0,0.2,0,0.2,0.25,0,0.25,0.25,0,0,0,0],[1,0.6666666667,0.25,0.6666666667,0,0.25,0,0.25,0,0,0.2,0.3333333333,0.5,0.25,0.2,0,0.25,0,0.3333333333,0.3333333333,1],[1,0.4,0.75,0.1666666667,0.2,0.1666666667,0.2,0.1666666667,0.2,0.2,0,0,0.3333333333,0.75,0.6,0.75,0.1666666667,0.25,0,0.5,0.2],[1,0.25,0,0.6666666667,0,0.6666666667,0,0.6666666667,0,0.3333333333,0,0,0.5,0,0.2,0,0.6666666667,0,1,0,0.3333333333],[1,0.75,0.1666666667,0.75,0.2,0.75,0.2,0.75,0.2,0.5,0.3333333333,0.5,0,0.4,0.3333333333,0.1666666667,0.75,0,0.5,0.2,0.5],[1,0.5,0.5,0.2,0.25,0.2,0.25,0.2,0.25,0.25,0.75,0,0.4,0,0.4,0.5,0.2,0.3333333333,0,0.6666666667,0.25],[1,0.1666666667,0.75,0.4,0,0.1666666667,0,0.1666666667,0,0.2,0.6,0.2,0.3333333333,0.4,0,0.4,0.1666666667,0.25,0.2,0.5,0.2],[1,0.2,0.5,0,0.25,0.2,0.25,0.2,0.25,0,0.75,0,0.1666666667,0.5,0.4,0,0.2,0.3333333333,0,0.25,0],[1,0.5,0,0.5,0.25,1,0.25,1,0.25,0.25,0.1666666667,0.6666666667,0.75,0.2,0.1666666667,0.2,0,0,0.6666666667,0,0.25],[1,0,0.3333333333,0,0,0,0,0,0,0,0.25,0,0,0.3333333333,0.25,0.3333333333,0,0,0,0.5,0],[1,0.25,0,0.6666666667,0,0.6666666667,0,0.6666666667,0,0.3333333333,0,1,0.5,0,0.2,0,0.6666666667,0,0,0,0.3333333333],[1,0.25,0.6666666667,0.25,0,0,0,0,0,0.3333333333,0.5,0,0.2,0.6666666667,0.5,0.25,0,0.5,0,0,0.3333333333],[1,0.6666666667,0.25,0.6666666667,0,0.25,0,0.25,0,1,0.2,0.3333333333,0.5,0.25,0.2,0,0.25,0,0.3333333333,0.3333333333,0]]
jc = []
for i, x in enumerate(X):
    for j in range(1, len(x)):
        jc.append(jaccard[i][x[j] + 1])
    for j in range(1, len(x)):
        for k in range(j + 1, len(x)):
            jc.append(jaccard[x[j] + len(data['Vs'])][x[k] + 1])
print(jc)
print('満足度', sum(jc) / len(jc))

[[0, 16], [1, 4, 6, 12, 18], [2, 1, 9, 14], [3], [4, 5, 7], [5, 0, 2, 8, 19], [6, 13], [7, 10, 11, 15, 17], [8], [9, 3]]
[0.25, 0.75, 0.75, 0.4, 0.2, 1, 0.2, 0, 0.2, 0, 0.6666666667, 0.25, 0.5, 0.6666666667, 0.75, 0.5, 0.75, 0.6666666667, 0.6666666667, 1, 1, 0.5, 0.6666666667, 0.6666666667, 0.5, 0.6666666667, 0.6666666667, 0.6666666667, 0.6666666667, 1, 0.25, 0.3333333333, 0.5, 0.6666666667, 0.3333333333, 0.5, 0.6666666667, 1, 0.75, 0.5, 0.6666666667, 0.5]
満足度 0.5674603174690476


In [ ]:
proposed05 = [[0, 12, 16, 18], [1, 4, 6, 10], [2, 9], [3], [4, 5], [5, 0, 8], [6, 7, 15], [7, 2, 11, 17, 19], [8, 3], [9, 1, 13, 14]]
intended = [[0, 5, 6, 12], [1, 4, 10, 18], [2, 9, 13], [3], [4, 0, 19], [5, 8], [6, 7, 15], [7, 2, 11, 16, 17], [8, 3], [9, 1, 14]]
TP = [1, 2, 1, 0, 1, 2, 3, 1, 2]
FN = [2, 1, 1, 2, 0, 0, 1, 0, 0]
FP = [2, 1, 0, 1, 1, 0, 1, 0, 1]
# proposed10 = [[0, 12, 18], [1, 2, 17, 19], [2, 9], [3], [4, 4, 6, 10], [5, 0, 8], [6, 16], [7, 5, 7, 11, 15], [8, 3], [9, 1, 13, 14]]
# intended = [[0, 5, 6, 12], [1, 4, 10, 18], [2, 9, 13], [3], [4, 0, 19], [5, 8], [6, 7, 15], [7, 2, 11, 16, 17], [8, 3], [9, 1, 14]]
# TP = [0, 2, 1, 0, 1, 0, 2, 0, 0]
# FN = [3, 1, 1, 2, 0, 2, 2, 1, 2]
# FP = [1, 2, 2, 2, 3, 1, 2, 0, 1]

recalls = []
precisions = []
accuracys = []
for i in range(len(TP)):
    if TP[i] + FN[i] > 0:
        recall = TP[i] / (TP[i] + FN[i])
    if TP[i] + FP[i] > 0:
        precision = TP[i] / (TP[i] + FP[i])
    if precision >= 0.5:
        accuracy = recall * (2 - 1 / precision)
        accuracys.append(accuracy)
    recalls.append(recall)
    precisions.append(precision)
    
print(recalls)
print(precisions)
print(accuracys)
print(sum(recalls) / len(recalls))
print(sum(precisions) / len(precisions))
print(sum(accuracys) / len(accuracys))